# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
F. Walter  ->  F. Walter  |  ['F. Walter']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
N. Wang  ->  N. Wang  |  ['N. Wang']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Li  ->  J. Li  |  ['J. Li']


A. Hughes  ->  A. Hughes  |  ['A. Hughes']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
Arxiv has 83 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.12129


extracting tarball to tmp_2506.12129...

 done.
  1: tmp_2506.12129/sample7.tex, 1,516 lines
  2: tmp_2506.12129/preamble.tex, 551 lines


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2506.12129/sample7.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'source_info_table' from 'tmp_2506.12129/source_info_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.

...

Found 82 bibliographic references in tmp_2506.12987/IMF.bbl.
Issues with the citations
syntax error in line 347: '=' expected
Retrieving document from  https://arxiv.org/e-print/2506.13618


extracting tarball to tmp_2506.13618...

 done.
Retrieving document from  https://arxiv.org/e-print/2506.13736


extracting tarball to tmp_2506.13736...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.12129-b31b1b.svg)](https://arxiv.org/abs/2506.12129) | **A 13-Billion-Year View of Galaxy Growth: Metallicity Gradient Evolution from the Local Universe to $z=9$ with JWST and Archival Surveys**  |
|| Z. Li, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-06-17*|
|*Comments*| *The main results are presented in Fig. 8. Comments are welcome*|
|**Abstract**|            The gas-phase metallicity gradients of galaxies have been extensively studied over the past four decades both in the local and high-redshift universe, as they trace the baryon cycle and growth of galaxies. With the unprecedented spatial resolution and sensitivity of the James Webb Space Telescope (JWST), it is now possible to measure metallicity and its radial gradients out to redshifts as high as $z = 9$. Here, we present a sample of 456 spectroscopically confirmed galaxies from redshifts $1.5 \lesssim z \lesssim 9$ that are spatially resolved on sub-kiloparsec (kpc) scales by deep JWST NIRCam or NIRISS Wide Field Slitless Spectroscopy (WFSS). Synthesizing these new JWST observations with legacy observations from the literature, we observe that at redshift $z > 5$, galaxy centers are more metal-rich, exhibiting negative metallicity gradients of $\sim-0.4$ dex kpc$^{-1}$. These gradients flatten over time, reaching near-zero around $z \approx 2$, coinciding with the peak of the cosmic star formation rate. Beyond this point, the gradients become negative again at lower redshifts approaching $z=0$. This evolution likely reflects transitions in galaxy formation modes: an inside-out growth phase dominated by intense central star formation with inefficient feedback and limited gas mixing during ``cosmic dawn", enhanced gas mixing due to feedback-driven wind and gas accretion at ``cosmic noon", and a later phase of slow evolution and reduced feedback toward the present day. These physical processes including gas accretion and feedback not only regulate star and galaxy formation on cosmic scale, but also shape the evolutionary pathways of individual galaxies over cosmic time.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.12136-b31b1b.svg)](https://arxiv.org/abs/2506.12136) | **The Star Formation in Radio Survey: Adding 90 GHz Data to 3-33 GHz Observations of Star-forming Regions in Nearby Galaxies**  |
|| A. Dignan, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-06-17*|
|*Comments*| *27 pages, 6 figures*|
|**Abstract**|            We present 90 GHz continuum imaging of 119 star-forming regions in 30 nearby galaxies observed with MUSTANG-2 on the Robert C. Byrd Green Bank Telescope as part of the Star Formation in Radio Survey. The 90 GHz data were combined with 3, 15, and 33 GHz data taken previously by the Karl G. Jansky Very Large Array to decompose radio spectra on $\approx$0.8 kpc scales into their synchrotron, free-free, and thermal dust emission components. This was done using three scenarios: (i) a power law fit from 3 to 33 GHz, (ii) Markov Chain Monte Carlo (MCMC) fitting from 3 to 90 GHz with a thermal dust component, and (iii) MCMC fitting from 3 to 33 GHz without a thermal dust component. For these cases, we find a median thermal (free-free) emission fraction at 33 GHz of (i) $88 \pm 2$% with a scatter of 17%, (ii) $76\pm 3$% with a scatter of 25%, and (iii) $84\pm 2$% with a scatter of 18%. From this we conclude that, on average, free-free, not thermal dust, remains the dominant emission component at 33 GHz. While scenario (ii) yields a thermal fraction that is $\approx$10% larger than scenario (iii), this difference decreases to $\approx$5% after AGN are removed. Consequently, star formation rates measured with thermal fractions at 33 GHz are only mildly biased high without 90 GHz data for the spectral decomposition. Furthermore, a power law fit of data from 3 to 33 GHz still provides a reliable estimate of the free-free emission at 33 GHz.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.12923-b31b1b.svg)](https://arxiv.org/abs/2506.12923) | **Simulating nearby disc galaxies on the main star formation sequence II. The gas structure transition in low and high stellar mass discs**  |
|| P. Verwilghen, et al. -- incl., <mark>J. Neumann</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-06-17*|
|*Comments*| *17 pages, 14 figures*|
|**Abstract**|            Recent hydrodynamical simulations of isolated barred disc galaxies have suggested a structural change in the distribution of the interstellar medium (ISM) around a stellar mass M$_{*}$ of $10^{10}$ M$_{\odot}$. In the higher-mass regime (M$_{*} \geq 10^{10}$ M$_{\odot}$), we observe the formation of a central gas and stellar disc with a typical size of a few hundred parsecs connected through lanes to the ends of the stellar bar. In the lower-mass regime (M$_{*} < 10^{10}$ M$_{\odot}$), such an inner disc is absent and the gas component exhibits a more chaotic distribution. Observations of nearby star-forming galaxies support the existence of such a change. These inner gas discs may represent an important intermediate scale connecting the large kiloparsec-scale structures with the nuclear (sub-parsec) region, transporting gas inwards to fuel the central supermassive black hole (SMBH). For this work, we used an extended set of high-resolution hydrodynamical simulations of isolated disc galaxies with initial properties (i.e. stellar mass, gas fraction, stellar disc scale length, and the bulge mass fraction) with properties covering the range of galaxies in the PHANGS sample to investigate this change of regime. We studied the physical properties of the star-forming ISM in both stellar mass regimes and extracted a few physical tracers: the inner Lindblad resonance (ILR), the probability distribution function (PDF), the virial parameter, and the Mach number. In line with observations, we confirm a structure transition in the simulations that occurs between a stellar mass of $10^{9.5}$ and $10^{10}$ M$_{\odot}$. We show that the physical origin of this change of regime is driven by stellar feedback and its contribution relative to the underlying gravitational potential.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.12987-b31b1b.svg)](https://arxiv.org/abs/2506.12987) | **Stellar initial mass function in the 100-pc solar neighbourhood**  |
|| Y.-T. Wang, C. Liu, <mark>J. Li</mark> |
|*Appeared on*| *2025-06-17*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**|            The stellar initial mass function (IMF) is one of the most important astrophysical distribution function, and is defined as the mass distribution for stars formed in a single star-formation event and when they just enter the main-sequence stage. The measurement of IMF is challenging even in the solar neighbourhood where IMF can be extracted by star-counting. This is because the definition of the IMF should be handled cautiously, and there are multiple corrections should be made from observation to a well-founded measurement. We present a new parametrisation of the stellar IMF in the 100-pc solar neighbourhood, leveraging the astrometric and photometric data from \textsl{Gaia} DR3: we model the colour-magnitude diagram of the field star population while accounting for observational uncertainties, Malmquist bias, Lutz-Kelker bias, the effect of varying mass-luminosity relation caused by mixture of different metallicity, and the effect of unresolved binaries. In particular, we synthesise the binaries with a process imitating the dynamical evolution observed in star clusters to model their present-day mass-ratio distribution. We derive a stellar IMF consistent with canonical IMFs but with significantly reduced uncertainties: $\alpha_1=0.81^{+0.06}_{-0.05}$, $\alpha_2=2.12^{+0.04}_{-0.04}$, and a break point at $m_{\mathrm{break}}=0.41^{+0.01}_{-0.01}$ $\mathrm{M_{\odot}}$. Our model also constrains the binary fraction to $\sim$25\% and the \textsl{Gaia} DR3 angular resolution to $1.31^{+0.24}_{-0.29}$ arcsec. We also provide the $\xi$-parameter for our IMF to be $0.5075_{-0.0051}^{+0.0112}$ for the users to compare our result with other work.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.12272-b31b1b.svg)](https://arxiv.org/abs/2506.12272) | **Polarization Studies of Black Widows PSRs B1957+20, J2055+3829 and J1544+4937**  |
|| S. Wang, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-06-17*|
|*Comments*| *8 pages, 3 figures, Accepted for publication in MNRAS*|
|**Abstract**|            We present an analysis of the polarization of three black widow pulsars, PSRs B1957+20, J2055+3829 and J1544+4937 at 1250\,MHz using the Five-hundred-meter Aperture Spherical radio Telescope (FAST). Radio eclipses for PSRs B1957+20 and J2055+3829 are detected, while the radio emission for J1544+4937 is detected throughout the eclipse. We study the polarization and dispersion measure properties of the pulsars near and during the eclipse. The position angle of the linear polarization is observed to shift at the eclipse boundary in all of these three pulsars implying a lower limit line-of-sight magnetic field strength of the eclipse medium of approximately a few mG to tens of mG. We also find evidence that the magnetic field in the eclipse medium of PSRs B1957+20 and J1544+4937 reverses.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.12294-b31b1b.svg)](https://arxiv.org/abs/2506.12294) | **Intense Hard X-ray Emissions in C-class Flares: A Statistical Study with ASO-S/HXI Data**  |
|| C. Chen, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-06-17*|
|*Comments*| *14 pages, 5 figures, Accepted in ApJL*|
|**Abstract**|            In the standard model of solar eruptive events, coronal mass ejections (CMEs) and flares are associated with each other through magnetic reconnection initiated by erupting flux ropes. Observations also reveal an increasing association ratio between flares and CMEs with flare intensity. However, the fundamental relationship between flares and CMEs, and that between thermal and nonthermal processes, remains unknown. Here we investigate energetic C-class flares (ECFs) -- Geostationary Operational Environmental Satellite (GOES) C-class flares with hard X-ray (HXR) emissions above 30 keV -- using observations from Advanced Space-based Solar Observatory/Hard X-ray Imager (HXI), Solar Dynamic Observatory, and GOES. Among 1331 C-class flares detected by HXI, 127 ECFs (9.5%) were identified for statistical analysis of their properties and associations with CMEs and other flare-related features. Our statistical results reveal that ECFs have relatively shorter durations and harder spectra (the mean electron power-law index is 4.65), with no significant correlation between soft X-ray flux and nonthermal parameters (e.g., HXR peak flux). Among the 127 events, 53 (42%) were associated with type III bursts, 38 (30%) with jets, at least 13 (~11%) with 360 nm brightenings, and only 5 (~4%) with CMEs. Crucially, all five CME events were narrow CMEs associated with jets. The surprising correlation between these ECFs and CMEs suggests that noneruptive or confined magnetic field configurations in these flares may favor electron acceleration, resulting in harder X-ray this http URL discuss the potential formation mechanisms and efficient electron acceleration processes in these atypical flares, providing valuable insights into nonstandard flare behavior.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.12387-b31b1b.svg)](https://arxiv.org/abs/2506.12387) | **The peculiar hard state behaviour of the black hole X-ray binary Swift J1727.8$-$1613**  |
|| A. K. Hughes, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-06-17*|
|*Comments*| *13 pages, 7 figures, 1 table including appendix, Submitted to MNRAS*|
|**Abstract**|            Tracking the correlation between radio and X-ray luminosities during black hole X-ray binary outbursts is a key diagnostic of the coupling between accretion inflows (traced by X-rays) and relativistic jet outflows (traced by radio). We present the radio--X-ray correlation of the black hole low-mass X-ray binary Swift~J1727.8$-$1613 during its 2023--2024 outburst. Our observations span a broad dynamic range, covering $\sim$4 orders of magnitude in radio luminosity and $\sim$6.5 in X-ray luminosity. This source follows an unusually radio-quiet track, exhibiting significantly lower radio luminosities at a given X-ray luminosity than both the standard (radio-loud) track and most previously known radio-quiet systems. Across most of the considered distance range ($D {\sim} 1.5--4.3$ kpc), Swift~J1727.8$-$1613 appears to be the most radio-quiet black hole binary identified to date. For distances ${\geq} 4$ kpc, while Swift~J1727.8$-$1613 becomes comparable to one other extremely radio-quiet system, its peak X-ray luminosity (${\gtrsim} 5{\times}10^{38}$ erg/s) exceeds that of any previously reported hard-state black hole low-mass X-ray binary, emphasising the extremity of this outburst. Additionally, for the first time in a radio-quiet system, we identify the onset of X-ray spectral softening to coincide with a change in trajectory through the radio--X-ray plane. We assess several proposed explanations for radio-quiet behaviour in black hole systems in light of this dataset. As with other such sources, however, no single mechanism fully accounts for the observed properties, highlighting the importance of regular monitoring and the value of comprehensive (quasi-)simultaneous datasets.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.13618-b31b1b.svg)](https://arxiv.org/abs/2506.13618) | **Thermal electrons in the radio afterglow of relativistic tidal disruption event ZTF22aaajecp/AT2022cmc**  |
|| L. Rhodes, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2025-06-17*|
|*Comments*| *18 pages, 8 figures, 1 table. Submitted to ApJ. Comments welcome*|
|**Abstract**|            A tidal disruption event (TDE) occurs when a star travels too close to a supermassive black hole. In some cases, accretion of the disrupted material onto the black hole launches a relativistic jet. In this paper, we present a long term observing campaign to study the radio and sub-millimeter emission associated with the fifth jetted/relativistic TDE: AT2022cmc. Our campaign reveals a long lived counterpart. We fit three different models to our data: a non-thermal jet, a spherical outflow consisting of both thermal and non-thermal electrons, and a jet with thermal and non-thermal electrons. We find that the data is best described by a relativistic spherical outflow propagating into an environment with a density profile following R^-1.8. Comparison of AT2022cmc to other TDEs finds agreement in the density profile of the environment but also that AT2022cmc is twice as energetic as the other well-studied relativistic TDE Swift J1644. Our observations of AT2022cmc allow a thermal electron population to be inferred for the first time in a jetted transient providing, new insights into the microphysics of relativistic transients jets.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.13736-b31b1b.svg)](https://arxiv.org/abs/2506.13736) | **DESI survey of S IV absorption outflows in quasars: Contribution to AGN feedback and comparison with [O III] emission outflows**  |
|| <mark>M. Sharma</mark>, et al. |
|*Appeared on*| *2025-06-17*|
|*Comments*| *9 pages, 7 figures. Accepted for Publication in A&A*|
|**Abstract**|            Quasar outflows can play a crucial role in the evolution of their host galaxies through various feedback processes. This effect is expected to be particularly important when the universe was only 2-3 billion years old, during the period known as cosmic noon. By utilizing observations from the Dark Energy Spectroscopy Instrument (DESI), we conduct a survey of high-ionization quasar outflows at cosmic noon, with the aim of doubling the current sample of such outflows with distance and energetics determination. We also aim to compare these properties to those derived from spatially resolved outflows in similar quasars probed through Integral Field Spectroscopy (IFS). In a sample of 130 quasars we detect signatures of high-ionization outflows in the form of a S IV trough in eight objects. We characterize the physical conditions of these outflows by determining their total hydrogen column density ($N_H$), ionization parameter ($U_H$), and electron number density ($n_e$). The detected outflows show complex kinematic structures with a wide range in blueshifted velocities (100$-$4600 km s$^{-1}$). We locate five out of the eight outflows at distances between 240$-$5500 pc away from the central source. From the combined sample of 15 high-ionization S IV outflows at cosmic noon, we find a high fraction (up to 46 %) of them to be powerful enough to contribute significantly to multi-stage AGN feedback processes. Their mass outflow rates and kinetic luminosities are also found to be consistent with spatially resolved outflows in a luminosity and redshift matched sample of quasars. Comparison with previous spectra reveals interesting variations in some objects, with two cases of emerging high-velocity broad absorption line features with velocities of $-$8000 and $-$39,000 km s$^{-1}$. An impressive case of four line-locked Si IV outflow systems is also revealed in one of the objects.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.12129/img/zgrad-z-obs.png', 'tmp_2506.12129/./img/ASPIRE_stacking.png', 'tmp_2506.12129/./img/FRESCO_stacking.png', 'tmp_2506.12129/./img/zgrad-m-fit.png']
copying  tmp_2506.12129/img/zgrad-z-obs.png to _build/html/
copying  tmp_2506.12129/./img/ASPIRE_stacking.png to _build/html/
copying  tmp_2506.12129/./img/FRESCO_stacking.png to _build/html/
copying  tmp_2506.12129/./img/zgrad-m-fit.png to _build/html/
exported in  _build/html/2506.12129.md
    + _build/html/tmp_2506.12129/img/zgrad-z-obs.png
    + _build/html/tmp_2506.12129/./img/ASPIRE_stacking.png
    + _build/html/tmp_2506.12129/./img/FRESCO_stacking.png
    + _build/html/tmp_2506.12129/./img/zgrad-m-fit.png
found figures ['tmp_2506.12136/./trends.png', 'tmp_2506.12136/./spectralindexhist_powerlaw.png', 'tmp_2506.12136/./f1_1.png']
copying  tmp_2506.12136/./trends.png to _build/html/
copying  tmp_2506.12136/./spectralindexhist_powerlaw.png to _build/html/
copying  tmp_2506.12136/./f1_1.png to _build/htm

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\kms}{\ensuremath{\rm km~s^{-1}}\xspace}$
$\newcommand{\lya}{\textrm{Ly}\ensuremath{\alpha}\xspace}$
$\newcommand{\secref}[1]{Section~\ref{#1}}$
$\newcommand{\figref}[1]{Figure~\ref{#1}}$
$\newcommand{\Eqref}[1]{Equation~(\ref{#1})}$
$\newcommand$
$\newcommand$
$\newcommand{\arcsec}{\mbox{^{\prime\prime}}\xspace}$
$\newcommand{\arcmin}{\mbox{\!^{\prime}}\xspace}$
$\newcommand{\arcdeg}{\mbox{^{\circ}}\xspace}$
$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\non}{\nonumber}$
$\newcommand{\ba}{\begin{align}}$
$\newcommand{\ea}{\end{align}}$
$\newcommand{\nt}{\notag}$
$\newcommand\abs{#1}{\left|{#1}\right|}$
$\newcommand{\avg}[1]{\left<#1\right>}$
$\newcommand{\newcommandeq}{\vcentcolon=}$
$\newcommand{\eqnewcommand}{=\vcentcolon}$
$\newcommand{\Ra}{\ensuremath{\Rightarrow}\xspace}$
$\newcommand{\ra}{\ensuremath{\rightarrow}\xpace}$
$\newcommand{\lra}{\ensuremath{\Leftrightarrow}\xspace}$
$\newcommand{\stmean}[1]{\langle{#1}\rangle}$
$\newcommand{\logg}{\log_{10}}$
$\DeclareMathOperator{\sk}{S}$
$\DeclareMathOperator{\tr}{tr}$
$\DeclareMathOperator{\ud}{d}$
$\newcommand{\Msun}{\ensuremath{M_\odot}\xspace}$
$\newcommand{\Rsun}{\ensuremath{R_\odot}\xspace}$
$\newcommand{\Lsun}{\ensuremath{L_\odot}\xspace}$
$\newcommand{\thE}{\ensuremath{\theta_{\rm E}}\xspace}$
$\newcommand{\chisq}{\ensuremath{\chi^2}\xspace}$
$\newcommand{\zspec}{\ensuremath{z_{\rm spec}}\xspace}$
$\newcommand{\zphot}{\ensuremath{z_{\rm phot}}\xspace}$
$\newcommand{\Mstar}{\ensuremath{M_\ast}\xspace}$
$\newcommand{\Lstar}{\ensuremath{L_\ast}\xspace}$
$\newcommand{\Sstar}{\ensuremath{\Sigma_\ast}\xspace}$
$\newcommand{\oh}{\ensuremath{12+\log({\rm O/H})}\xspace}$
$\newcommand{\Av}{\ensuremath{A_{\rm V}}\xspace}$
$\newcommand{\Rv}{\ensuremath{R_{\rm V}}\xspace}$
$\newcommand{\Te}{\ensuremath{T_{\rm e}}\xspace}$
$\newcommand{\ne}{\ensuremath{n_{\rm e}}\xspace}$
$\newcommand{\SFR}{\ensuremath{{\rm SFR}}\xspace}$
$\newcommand{\Mgas}{\ensuremath{M_{\rm gas}}\xspace}$
$\newcommand{\Sgas}{\ensuremath{\Sigma_{\rm gas}}\xspace}$
$\newcommand{\fgas}{\ensuremath{f_{\rm gas}}\xspace}$
$\newcommand{\Zgas}{\ensuremath{Z_{\rm gas}}\xspace}$
$\newcommand{\tage}{\ensuremath{t_{\rm age}}\xspace}$
$\newcommand{\Vrot}{\ensuremath{V_{\rm rot}}\xspace}$
$\newcommand{\reff}{\ensuremath{r_{\rm eff}}\xspace}$
$\newcommand{\Dn}{\ensuremath{{\rm D}_n(4000)}\xspace}$
$\newcommand{\HdA}{\ensuremath{{\rm H}\delta_A}\xspace}$
$\newcommand{\scrit}{\ensuremath{\sigma_{\rm crit}}\xspace}$
$\newcommand{\fesc}{\ensuremath{f_{\rm esc}}\xspace}$
$\newcommand{\xion}{\ensuremath{\xi_{\rm ion}}\xspace}$
$\newcommand{\tauIGM}{\ensuremath{\tau_{\rm IGM}}\xspace}$
$\newcommand{\Muv}{\ensuremath{M_{\rm UV}}\xspace}$
$\newcommand{\eV}{\ensuremath{\rm eV}\xspace}$
$\newcommand{\pc}{\ensuremath{\rm pc}\xspace}$
$\newcommand{\kpc}{\ensuremath{\rm kpc}\xspace}$
$\newcommand{\Mpc}{\ensuremath{\rm Mpc}\xspace}$
$\newcommand{\K}{\ensuremath{\rm K}\xspace}$
$\newcommand{\mK}{\ensuremath{\rm mK}\xspace}$
$\newcommand{\Hunit}{\ensuremath{\rm km~s^{-1}~Mpc^{-1}}\xspace}$
$\newcommand{\Funit}{\ensuremath{\rm erg~s^{-1}~cm^{-2}}\xspace}$
$\newcommand{\Flam}{\ensuremath{\rm erg~s^{-1}~cm^{-2}~Å^{-1}}\xspace}$
$\newcommand{\Fnu}{\ensuremath{\rm erg~s^{-1}~cm^{-2}~Hz^{-1}}\xspace}$
$\newcommand{\muJy}{\ensuremath{\mu\rm Jy}\xspace}$
$\newcommand{\SBunit}{\ensuremath{\rm erg~s^{-1}~cm^{-2}~arcsec^{-2}}\xspace}$
$\newcommand{\magarcs}{\ensuremath{\rm mag~arcsec^{-2}}\xspace}$
$\newcommand{\Msunyr}{\ensuremath{\Msun~\mathrm{yr}^{-1}}\xspace}$
$\newcommand{\yr}{\ensuremath{\rm yr}\xspace}$
$\newcommand{\Myr}{\ensuremath{\rm Myr}\xspace}$
$\newcommand{\Gyr}{\ensuremath{\rm Gyr}\xspace}$
$\newcommand{\micron}{\ensuremath{\mu\textrm{m}}\xspace}$
$\newcommand\ionp[2]{#1\;{\scshape{#2}}}$
$\newcommand\ionf[2]{[#1\;{\scshape{#2}}]}$
$\newcommand\ions[2]{#1\;{\scshape{#2}}]}$
$\newcommand{\Ha}{\textrm{H}\ensuremath{\alpha}\xspace}$
$\newcommand{\Hb}{\textrm{H}\ensuremath{\beta}\xspace}$
$\newcommand{\Hg}{\textrm{H}\ensuremath{\gamma}\xspace}$
$\newcommand{\Hd}{\textrm{H}\ensuremath{\delta}\xspace}$
$\newcommand{\HII}{\textrm{H}\textsc{ii}\xspace}$
$\newcommand{\HI}{\textrm{H}\textsc{i}\xspace}$
$\newcommand{\Htwo}{\textrm{H}\ensuremath{_2}\xspace}$
$\newcommand{\He}{\textrm{He}\xspace}$
$\newcommand{\OI}{[\textrm{O}~\textsc{i}]\xspace}$
$\newcommand{\OII}{[\textrm{O}~\textsc{ii}]\xspace}$
$\newcommand{\OIII}{[\textrm{O}~\textsc{iii}]\xspace}$
$\newcommand{\CIII}{\textrm{C}~\textsc{iii}]\xspace}$
$\newcommand{\NII}{[\textrm{N}~\textsc{ii}]\xspace}$
$\newcommand{\SII}{[\textrm{S}~\textsc{ii}]\xspace}$
$\newcommand{\NeIII}{[\textrm{Ne}~\textsc{iii}]\xspace}$
$\newcommand{\HeII}{\textrm{He}~\textsc{ii}\xspace}$
$\newcommand{\CIV}{\textrm{C}~\textsc{iv}\xspace}$
$\newcommand{\SiIV}{\textrm{Si}~\textsc{iv}\xspace}$
$\newcommand{\CII}{\textrm{C}~\textsc{ii}\xspace}$
$\newcommand{\sersic}{Sérsic\xspace}$
$\newcommand{\U}{\ensuremath{U_{360}}\xspace}$
$\newcommand{\B}{\ensuremath{B_{435}}\xspace}$
$\newcommand{\V}{\ensuremath{V_{606}}\xspace}$
$\newcommand{\I}{\ensuremath{I_{814}}\xspace}$
$\newcommand{\Y}{\ensuremath{Y_{105}}\xspace}$
$\newcommand{\J}{\ensuremath{J_{125}}\xspace}$
$\newcommand{\JH}{\ensuremath{JH_{140}}\xspace}$
$\newcommand{\H}{\ensuremath{H_{160}}\xspace}$
$\newcommand{\clyi}{MACS1149.6+2223\xspace}$
$\newcommand{\cler}{Abell 2744\xspace}$
$\newcommand{\clsan}{Abell 370\xspace}$
$\newcommand{\clsi}{MACS0416.1-2403\xspace}$
$\newcommand{\clwu}{MACS0717.5+3745\xspace}$
$\newcommand{\clliu}{RXJ2248.7-4431\xspace}$
$\newcommand{\clqi}{RXJ1347.5-1145\xspace}$
$\newcommand{\clba}{MACS0744.9+3927\xspace}$
$\newcommand{\cljiu}{MACS2129.4-0741\xspace}$
$\newcommand{\clshi}{MACS1423.8+2404\xspace}$
$\newcommand{\pylf}{\textsc{pyLensFix}\xspace}$
$\newcommand{\lf}{\textsc{LensFix}\xspace}$
$\newcommand{\sw}{\textsc{SWunited}\xspace}$
$\newcommand{\sex}{\textsc{SExtractor}\xspace}$
$\newcommand{\emc}{\textsc{Emcee}\xspace}$
$\newcommand{\linmix}{\textsc{linmix}\xspace}$
$\newcommand{\adriz}{\textsc{AstroDrizzle}\xspace}$
$\newcommand{\dpac}{\textsc{DrizzlePac}\xspace}$
$\newcommand{\fast}{\textsc{FAST}\xspace}$
$\newcommand{\galfit}{\textsc{Galfit}\xspace}$
$\newcommand{\axe}{\textsc{aXe}\xspace}$
$\newcommand{\lt}{\textsc{Lenstool}\xspace}$
$\newcommand{\glafic}{\textsc{Glafic}\xspace}$
$\newcommand{\gasoline}{\textsc{Gasoline}\xspace}$
$\newcommand{\ramses}{\textsc{Ramses}\xspace}$
$\newcommand{\SJ}{\textsc{Sharon \& Johnson}\xspace}$
$\newcommand{\grzl}{\textsc{Grizli}\xspace}$
$\newcommand{\burst}{\textsc{Starburst99}\xspace}$
$\newcommand{\tphot}{\textsc{T-PHOT}\xspace}$
$\newcommand{\bagp}{\textsc{BAGPIPES}\xspace}$
$\newcommand{\phut}{\textsc{Photutils}\xspace}$
$\newcommand{\msa}{\textsc{msaexp}\xspace}$
$\newcommand{\ppxf}{\textsc{pPXF}\xspace}$
$\newcommand{\jwst}{\textit{JWST}\xspace}$
$\newcommand{\planck}{\textit{Planck}\xspace}$
$\newcommand{\hst}{\textit{HST}\xspace}$
$\newcommand{\hubble}{\textit{Hubble}\xspace}$
$\newcommand{\spitzer}{\textit{Spitzer}\xspace}$
$\newcommand{\herschel}{\textit{Herschel}\xspace}$
$\newcommand{\chandra}{\textit{Chandra}\xspace}$
$\newcommand{\glass}{\textit{GLASS}\xspace}$
$\newcommand{\clear}{\textit{CLEAR}\xspace}$
$\newcommand{\wisp}{\textit{WISP}\xspace}$
$\newcommand{\clash}{\textit{CLASH}\xspace}$
$\newcommand{\candels}{\textit{CANDELS}\xspace}$
$\newcommand{\uvc}{\textit{UVCANDELS}\xspace}$
$\newcommand{\hff}{\textit{HFF}\xspace}$
$\newcommand{\muse}{\textit{MUSE}\xspace}$
$\newcommand{\kmos}{\textit{KMOS}\xspace}$
$\newcommand{\keck}{\textit{Keck}\xspace}$
$\newcommand{\deimos}{\textit{DEIMOS}\xspace}$
$\newcommand{\mosfire}{\textit{MOSFIRE}\xspace}$
$\newcommand{\surfsup}{\textit{SURFSUP}\xspace}$
$\newcommand{\kd}{\textit{KMOS}^{3\rm D}\xspace}$
$\newcommand{\sdss}{\textit{SDSS}\xspace}$
$\newcommand{\clash}{\textit{CLASH}\xspace}$
$\newcommand{\mosnewcommand}{\textit{MOSDEF}\xspace}$
$\newcommand{\vlt}{\textit{VLT}\xspace}$
$\newcommand{\osiris}{\textit{OSIRIS}\xspace}$
$\newcommand{\sinf}{\textit{SINFONI}\xspace}$
$\newcommand{\ngrst}{\textit{NGRST}\xspace}$
$\newcommand{\niriss}{\textit{NIRISS}\xspace}$
$\newcommand{\mmth}{\textit{MAMMOTH}\xspace}$
$\newcommand{\mg}{\textit{MAMMOTH-Grism}\xspace}$
$\newcommand{\etal}{et al.\xspace}$
$\newcommand{\ie}{i.e.\xspace}$
$\newcommand{\eg}{e.g.\xspace}$
$\newcommand{\etc}{etc.\xspace}$
$\newcommand{\aka}{a.k.a.\xspace}$
$\newcommand{\vsv}{vis-á-vis\xspace}$
$\newcommand\({\left(}$
$\newcommand\){\right)}$
$\newcommand\sfr{star-formation rate\xspace}$
$\newcommand\sfh{star-formation history\xspace}$
$\newcommand\sfms{star-formation main sequence\xspace}$
$\newcommand{\el}[1]{\ensuremath{\textrm{EL}_{#1}}}$
$\newcommand{\obs}{\textrm{o}}$
$\newcommand{\theo}{\textrm{t}}$
$\newcommand{\ext}{\textrm{ext}}$
$\newcommand{\det}{\textrm{det}}$
$\newcommand\refe{\textrm{ref}}$
$\newcommand\pa{\textrm{PA}}$
$\newcommand{\p}{{\rm prior}}$
$\newcommand{\fid}{{\rm fid}}$
$\newcommand{\lnk}{\kappa}$
$\newcommand{\lnkp}{\kappa'}$
$\newcommand{\xa}{\alpha}$
$\newcommand{\xb}{\beta}$
$\newcommand{\xk}{\kappa}$
$\newcommand{\xg}{\gamma}$
$\newcommand{\Or}{\ensuremath{\Omega_{\rm{r}}}\xspace}$
$\newcommand{\Om}{\ensuremath{\Omega_{\rm{m}}}\xspace}$
$\newcommand{\Ok}{\ensuremath{\Omega_{\rm{k}}}\xspace}$
$\newcommand{\Ol}{\ensuremath{\Omega_{\Lambda}}\xspace}$
$\newcommand{\Obh}{\ensuremath{\Omega_{\rm{b}}h^2}\xspace}$
$\newcommand{\Ob}{\ensuremath{\Omega_{\rm{b}}}\xspace}$
$\newcommand{\Onu}{\ensuremath{\Omega_\nu}\xspace}$
$\newcommand{\fnu}{\ensuremath{f_{\nu}}\xspace}$
$\newcommand{\Och}{\ensuremath{\Omega_{\rm{DM}}h^2}\xspace}$
$\newcommand{\Oc}{\ensuremath{\Omega_{\rm{DM}}}\xspace}$
$\newcommand{\ns}{\ensuremath{n_{\rm s}}\xspace}$
$\newcommand{\As}{\ensuremath{A_{\rm s}}\xspace}$
$\newcommand{\thA}{\ensuremath{\theta_{\rm A}}\xspace}$
$\newcommand{\neff}{\ensuremath{N_\textrm{eff}}\xspace}$
$\newcommand{\mnu}{\ensuremath{\sum{m_{\nu}}}\xspace}$
$\newcommand{\yhe}{\ensuremath{Y_p}\xspace}$
$\newcommand{\Map}[1]{\left<M^2_\textrm{ap}\right>( #1 )}$
$\newcommand{\map}{\ensuremath{\left<M^2_\textrm{ap}\right>}\xspace}$
$\newcommand{\chiH}{\ensuremath{\chi_\textrm{H}}\xspace}$
$\newcommand{\n}{\ensuremath{{\nu}\rm}\xspace}$
$\newcommand{\nue}{\ensuremath{{\nu}_{\rm e}}\xspace}$
$\newcommand{\num}{\ensuremath{{\nu}_{\rm \mu}}\xspace}$
$\newcommand{\nut}{\ensuremath{{\nu}_{\rm \tau}}\xspace}$
$\newcommand{\da}{\ensuremath{D_{\rm A}}\xspace}$
$\newcommand{\dl}{\ensuremath{D_{\rm L}}\xspace}$
$\newcommand{\taueq}{\ensuremath{\tau_{\rm eq}}\xspace}$
$\newcommand{\hi}{H \textsc{i}}$
$\newcommand{\heii}{He \textsc{ii}}$
$\newcommand{\civ}{C \textsc{iv}}$
$\newcommand{\ciii}{[C \textsc{iii}]}$
$\newcommand{\oii}{O \textsc{ii}}$
$\newcommand{\oiii}{O \textsc{iii}}$
$\newcommand{\ovi}{O \textsc{vi}}$
$\newcommand{\Ha}{\textrm{H}\ensuremath{\alpha}\xspace}$
$\newcommand{\Hb}{\textrm{H}\ensuremath{\beta}\xspace}$
$\newcommand{\Hg}{\textrm{H}\ensuremath{\gamma}\xspace}$
$\newcommand{\Hd}{\textrm{H}\ensuremath{\delta}\xspace}$
$\newcommand{\OII}{[\textrm{O}~\textsc{ii}]\xspace}$
$\newcommand{\OIII}{[\textrm{O}~\textsc{iii}]\xspace}$
$\newcommand{\NII}{[\textrm{N}~\textsc{ii}]\xspace}$
$\newcommand{\red}{\color{red}}$
$\newcommand{\blue}{\color{blue}}$
$\newcommand{\logm}{\log(M_*/M_\odot)}$
$\usepackage{graphicx}$
$\usepackage{xspace}$
$\usepackage{subcaption}$
$\usepackage{amsmath}$
$\usepackage{multirow}$
$\usepackage{pgffor}$
$\newcommand{\oh}{\ensuremath{12+\log({\rm O/H})}\xspace}$
$\newcommand{\el}[1]{\ensuremath{\textrm{EL}_{#1}}}$
$\newcommand{\Msun}{\ensuremath{M_\odot}\xspace}$
$\newcommandcitealias{Erb_08}{Erb08}$
$\graphicspath{{./}{img/}}$
$\usepackage[flushleft]{threeparttable}$
$\begin{document}$
$\title{A 13-Billion-Year View of Galaxy Growth: Metallicity Gradient Evolution from the Local Universe to z=9 with JWST and Archival Surveys}$
$\author[0000-0001-5951-459X]{Zihao Li}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\affiliation{Cosmic Dawn Center (DAWN), Denmark}$
$\affiliation{Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Copenhagen N, Denmark}$
$\email[show]{zihao.li@nbi.ku.dk}$
$\author[0000-0001-8467-6478]{Zheng Cai}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\email[show]{zcai@tsinghua.edu.cn}$
$\author[0000-0002-9373-3865]{Xin Wang}$
$\affiliation{School of Astronomy and Space Science, University of Chinese Academy of Sciences, Beijing 100049, China}$
$\affiliation{National Astronomical Observatories, Chinese Academy of Sciences, Beijing 100101, China}$
$\affiliation{Institute for Frontiers in Astronomy and Astrophysics, Beijing Normal University, Beijing 102206, China}$
$\email[show]{xwang@ucas.ac.cn}$
$\author[0000-0001-7890-4964]{Zhaozhou Li}$
$\affiliation{Center for Astrophysics and Planetary Science, Racah Institute of Physics, The Hebrew University, Jerusalem, 91904, Israel}$
$\email $
$\author[0000-0003-4174-0374]{Avishai Dekel}$
$\affiliation{Center for Astrophysics and Planetary Science, Racah Institute of Physics, The Hebrew University, Jerusalem, 91904, Israel}$
$\affiliation{Santa Cruz Institute for Particle Physics, University of California, Santa Cruz, CA 95064, USA}$
$\email $
$\author[0000-0002-7767-8472]{Kartick C. Sarkar}$
$\affiliation{Center for Astrophysics and Planetary Science, Racah Institute of Physics, The Hebrew University, Jerusalem, 91904, Israel}$
$\affiliation{Raman Research Institute, Sadashivanagar, C. V. Raman Avenue, 560 080, Bangalore, India}$
$\email $
$\author[0000-0002-2931-7824]{Eduardo Bañados}$
$\affiliation{Max-Plank-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\email $
$\author[0000-0002-1620-0897]{Fuyan Bian}$
$\affiliation{European Southern Observatory, Alonso de Cordova 3107, Casilla 19001, Vitacura, Santiago 19, Chile}$
$\email $
$\author[0000-0002-7080-2864]{Aklant K. Bhowmick}$
$\affiliation{Department of Physics, University of Florida, Gainesville, Florida 32601, USA}$
$\email $
$\author[0000-0002-2183-1087]{Laura Blecha}$
$\affiliation{Department of Physics, University of Florida, Gainesville, Florida 32601, USA}$
$\email $
$\author[0000-0001-8582-7012]{Sarah E. I. Bosman}$
$\affiliation{Max-Plank-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\affiliation{Institute for Theoretical Physics, Heidelberg University, Philosophenweg 12, D–69120, Heidelberg, Germany}$
$\email $
$\author[0000-0002-6184-9097]{Jaclyn B. Champagne}$
$\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Avenue, Tucson, AZ 85721, USA}$
$\email $
$\author[0000-0002-6336-3007]{Xiaohui Fan}$
$\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Avenue, Tucson, AZ 85721, USA}$
$\email $
$\author[0000-0001-5160-6713]{Emmet Golden-Marx}$
$\affiliation{INAF-Astronomical Observatory of Padova vicolo dell’Osservatorio 5 35122 Padova, Italy}$
$\email $
$\author[0000-0003-1470-5901]{Hyunsung D. Jun}$
$\affiliation{Department of Physics, Northwestern College, 101 7th Street SW, Orange City, Iowa 51041, USA}$
$\email $
$\author[0000-0001-6251-649X]{Mingyu Li}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\email $
$\author[0000-0001-6052-4234]{Xiaojing Lin}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Avenue, Tucson, AZ 85721, USA}$
$\email $
$\author[0000-0003-3762-7344]{Weizhe Liu}$
$\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Avenue, Tucson, AZ 85721, USA}$
$\email $
$\author[0000-0002-4622-6617]{Fengwu Sun}$
$\affiliation{Center for Astrophysics | Harvard \& Smithsonian, 60 Garden St., Cambridge, MA 02138, USA}$
$\email $
$\author[0000-0002-6849-5375]{Maxime Trebitsch}$
$\affiliation{LUX, Observatoire de Paris, Université PSL, Sorbonne Université, CNRS, 75014 Paris, France}$
$\email $
$\author[0000-0003-4793-7880]{Fabian Walter}$
$\affiliation{Max-Plank-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\email $
$\author[0000-0002-7633-431X]{Feige Wang}$
$\affiliation{Department of Astronomy, University of Michigan, 500 S State St, Ann Arbor, MI 48109, USA}$
$\email $
$\author[0000-0003-0111-8249]{Yunjing Wu}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\email $
$\author[0000-0001-5287-4242]{Jinyi Yang}$
$\affiliation{Department of Astronomy, University of Michigan, 500 S State St, Ann Arbor, MI 48109, USA}$
$\email $
$\author[0000-0002-0123-9246]{Huanian Zhang}$
$\affiliation{Department of Astronomy, Huazhong University of Science and Technology, Wuhan 430074, China}$
$\email $
$\author[0000-0002-0427-9577]{Shiwu Zhang}$
$\affiliation{Zhejiang Lab, Hangzhou, Zhejiang 311121, China}$
$\email $
$\author[0000-0001-5105-2837]{Mingyang Zhuang}$
$\affiliation{Department of Astronomy, University of Illinois at Urbana-Champaign, Urbana, IL 61801, USA}$
$\email $
$\author[0000-0002-3983-6484]{Siwei Zou}$
$\affiliation{Chinese Academy of Sciences South America Center for Astronomy, National Astronomical Observatories, CAS, Beijing 100101, China}$
$\affiliation{Departamento de Astronomía, Universidad de Chile, Casilla 36-D, Santiago, Chile}$
$\email $
$\begin{abstract}$
$The gas-phase metallicity gradients of galaxies have been extensively studied over the past four decades both in the local and high-redshift universe, as they trace the baryon cycle and growth of galaxies. With the unprecedented spatial resolution and sensitivity of the James Webb Space Telescope (JWST), it is now possible to measure metallicity and its radial gradients out to redshifts as high as z = 9. Here, we present a sample of 456 spectroscopically confirmed galaxies from redshifts 1.5 \lesssim z \lesssim 9 that are spatially resolved on sub-kiloparsec (kpc) scales by deep JWST NIRCam or NIRISS Wide Field Slitless Spectroscopy (WFSS). Synthesizing these new JWST observations with legacy observations from the literature, we observe that at redshift z > 5, galaxy centers are more metal-rich, exhibiting negative metallicity gradients of \sim-0.4 dex kpc^{-1}. These gradients flatten over time, reaching near-zero around z \approx 2, coinciding with the peak of the cosmic star formation rate. Beyond this point, the gradients become negative again at lower redshifts approaching z=0.$
$This evolution likely reflects transitions in galaxy formation modes: an inside-out growth phase dominated by intense central star formation with inefficient feedback and limited gas mixing during "cosmic dawn", enhanced gas mixing due to feedback-driven wind and gas accretion at "cosmic noon", and a later phase of slow evolution and reduced feedback toward the present day. These physical processes including gas accretion and feedback not only regulate star and galaxy formation on cosmic scale, but also shape the evolutionary pathways of individual galaxies over cosmic time.$
$\end{abstract}$
$\keywords{\uat{Galaxies}{573} --- \uat{High-redshift galaxies}{734} --- \uat{Chemical enrichment}{225} --- \uat{Galaxy evolution}{594} --- \uat{Galaxy formation}{595} --- \uat{Metallicity}{1031}}$
$\n\end{document}\end{align}}\end{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\non}{\nonumber}$
$\newcommand{\ba}{\begin{align}}$
$\newcommand{\ea}{\end{align}}$
$\newcommand{\nt}{\notag}$
$\newcommand\abs{#1}{\left|{#1}\right|}$
$\newcommand{\avg}[1]{\left<#1\right>}$
$\newcommand{\newcommandeq}{\vcentcolon=}$
$\newcommand{\eqnewcommand}{=\vcentcolon}$
$\newcommand{\Ra}{\ensuremath{\Rightarrow}\xspace}$
$\newcommand{\ra}{\ensuremath{\rightarrow}\xpace}$
$\newcommand{\lra}{\ensuremath{\Leftrightarrow}\xspace}$
$\newcommand{\stmean}[1]{\langle{#1}\rangle}$
$\newcommand{\logg}{\log_{10}}$
$\DeclareMathOperator{\sk}{S}$
$\DeclareMathOperator{\tr}{tr}$
$\DeclareMathOperator{\ud}{d}$
$\newcommand{\Msun}{\ensuremath{M_\odot}\xspace}$
$\newcommand{\Rsun}{\ensuremath{R_\odot}\xspace}$
$\newcommand{\Lsun}{\ensuremath{L_\odot}\xspace}$
$\newcommand{\thE}{\ensuremath{\theta_{\rm E}}\xspace}$
$\newcommand{\chisq}{\ensuremath{\chi^2}\xspace}$
$\newcommand{\zspec}{\ensuremath{z_{\rm spec}}\xspace}$
$\newcommand{\zphot}{\ensuremath{z_{\rm phot}}\xspace}$
$\newcommand{\Mstar}{\ensuremath{M_\ast}\xspace}$
$\newcommand{\Lstar}{\ensuremath{L_\ast}\xspace}$
$\newcommand{\Sstar}{\ensuremath{\Sigma_\ast}\xspace}$
$\newcommand{\oh}{\ensuremath{12+\log({\rm O/H})}\xspace}$
$\newcommand{\Av}{\ensuremath{A_{\rm V}}\xspace}$
$\newcommand{\Rv}{\ensuremath{R_{\rm V}}\xspace}$
$\newcommand{\Te}{\ensuremath{T_{\rm e}}\xspace}$
$\newcommand{\ne}{\ensuremath{n_{\rm e}}\xspace}$
$\newcommand{\SFR}{\ensuremath{{\rm SFR}}\xspace}$
$\newcommand{\Mgas}{\ensuremath{M_{\rm gas}}\xspace}$
$\newcommand{\Sgas}{\ensuremath{\Sigma_{\rm gas}}\xspace}$
$\newcommand{\fgas}{\ensuremath{f_{\rm gas}}\xspace}$
$\newcommand{\Zgas}{\ensuremath{Z_{\rm gas}}\xspace}$
$\newcommand{\tage}{\ensuremath{t_{\rm age}}\xspace}$
$\newcommand{\Vrot}{\ensuremath{V_{\rm rot}}\xspace}$
$\newcommand{\reff}{\ensuremath{r_{\rm eff}}\xspace}$
$\newcommand{\Dn}{\ensuremath{{\rm D}_n(4000)}\xspace}$
$\newcommand{\HdA}{\ensuremath{{\rm H}\delta_A}\xspace}$
$\newcommand{\scrit}{\ensuremath{\sigma_{\rm crit}}\xspace}$
$\newcommand{\fesc}{\ensuremath{f_{\rm esc}}\xspace}$
$\newcommand{\xion}{\ensuremath{\xi_{\rm ion}}\xspace}$
$\newcommand{\tauIGM}{\ensuremath{\tau_{\rm IGM}}\xspace}$
$\newcommand{\Muv}{\ensuremath{M_{\rm UV}}\xspace}$
$\newcommand{\eV}{\ensuremath{\rm eV}\xspace}$
$\newcommand{\pc}{\ensuremath{\rm pc}\xspace}$
$\newcommand{\kpc}{\ensuremath{\rm kpc}\xspace}$
$\newcommand{\Mpc}{\ensuremath{\rm Mpc}\xspace}$
$\newcommand{\K}{\ensuremath{\rm K}\xspace}$
$\newcommand{\mK}{\ensuremath{\rm mK}\xspace}$
$\newcommand{\Hunit}{\ensuremath{\rm km~s^{-1}~Mpc^{-1}}\xspace}$
$\newcommand{\Funit}{\ensuremath{\rm erg~s^{-1}~cm^{-2}}\xspace}$
$\newcommand{\Flam}{\ensuremath{\rm erg~s^{-1}~cm^{-2}~Å^{-1}}\xspace}$
$\newcommand{\Fnu}{\ensuremath{\rm erg~s^{-1}~cm^{-2}~Hz^{-1}}\xspace}$
$\newcommand{\muJy}{\ensuremath{\mu\rm Jy}\xspace}$
$\newcommand{\SBunit}{\ensuremath{\rm erg~s^{-1}~cm^{-2}~arcsec^{-2}}\xspace}$
$\newcommand{\magarcs}{\ensuremath{\rm mag~arcsec^{-2}}\xspace}$
$\newcommand{\Msunyr}{\ensuremath{\Msun~\mathrm{yr}^{-1}}\xspace}$
$\newcommand{\yr}{\ensuremath{\rm yr}\xspace}$
$\newcommand{\Myr}{\ensuremath{\rm Myr}\xspace}$
$\newcommand{\Gyr}{\ensuremath{\rm Gyr}\xspace}$
$\newcommand{\micron}{\ensuremath{\mu\textrm{m}}\xspace}$
$\newcommand\ionp[2]{#1\;{\scshape{#2}}}$
$\newcommand\ionf[2]{[#1\;{\scshape{#2}}]}$
$\newcommand\ions[2]{#1\;{\scshape{#2}}]}$
$\newcommand{\Ha}{\textrm{H}\ensuremath{\alpha}\xspace}$
$\newcommand{\Hb}{\textrm{H}\ensuremath{\beta}\xspace}$
$\newcommand{\Hg}{\textrm{H}\ensuremath{\gamma}\xspace}$
$\newcommand{\Hd}{\textrm{H}\ensuremath{\delta}\xspace}$
$\newcommand{\HII}{\textrm{H}\textsc{ii}\xspace}$
$\newcommand{\HI}{\textrm{H}\textsc{i}\xspace}$
$\newcommand{\Htwo}{\textrm{H}\ensuremath{_2}\xspace}$
$\newcommand{\He}{\textrm{He}\xspace}$
$\newcommand{\OI}{[\textrm{O}~\textsc{i}]\xspace}$
$\newcommand{\OII}{[\textrm{O}~\textsc{ii}]\xspace}$
$\newcommand{\OIII}{[\textrm{O}~\textsc{iii}]\xspace}$
$\newcommand{\CIII}{\textrm{C}~\textsc{iii}]\xspace}$
$\newcommand{\NII}{[\textrm{N}~\textsc{ii}]\xspace}$
$\newcommand{\SII}{[\textrm{S}~\textsc{ii}]\xspace}$
$\newcommand{\NeIII}{[\textrm{Ne}~\textsc{iii}]\xspace}$
$\newcommand{\HeII}{\textrm{He}~\textsc{ii}\xspace}$
$\newcommand{\CIV}{\textrm{C}~\textsc{iv}\xspace}$
$\newcommand{\SiIV}{\textrm{Si}~\textsc{iv}\xspace}$
$\newcommand{\CII}{\textrm{C}~\textsc{ii}\xspace}$
$\newcommand{\sersic}{Sérsic\xspace}$
$\newcommand{\U}{\ensuremath{U_{360}}\xspace}$
$\newcommand{\B}{\ensuremath{B_{435}}\xspace}$
$\newcommand{\V}{\ensuremath{V_{606}}\xspace}$
$\newcommand{\I}{\ensuremath{I_{814}}\xspace}$
$\newcommand{\Y}{\ensuremath{Y_{105}}\xspace}$
$\newcommand{\J}{\ensuremath{J_{125}}\xspace}$
$\newcommand{\JH}{\ensuremath{JH_{140}}\xspace}$
$\newcommand{\H}{\ensuremath{H_{160}}\xspace}$
$\newcommand{\clyi}{MACS1149.6+2223\xspace}$
$\newcommand{\cler}{Abell 2744\xspace}$
$\newcommand{\clsan}{Abell 370\xspace}$
$\newcommand{\clsi}{MACS0416.1-2403\xspace}$
$\newcommand{\clwu}{MACS0717.5+3745\xspace}$
$\newcommand{\clliu}{RXJ2248.7-4431\xspace}$
$\newcommand{\clqi}{RXJ1347.5-1145\xspace}$
$\newcommand{\clba}{MACS0744.9+3927\xspace}$
$\newcommand{\cljiu}{MACS2129.4-0741\xspace}$
$\newcommand{\clshi}{MACS1423.8+2404\xspace}$
$\newcommand{\pylf}{\textsc{pyLensFix}\xspace}$
$\newcommand{\lf}{\textsc{LensFix}\xspace}$
$\newcommand{\sw}{\textsc{SWunited}\xspace}$
$\newcommand{\sex}{\textsc{SExtractor}\xspace}$
$\newcommand{\emc}{\textsc{Emcee}\xspace}$
$\newcommand{\linmix}{\textsc{linmix}\xspace}$
$\newcommand{\adriz}{\textsc{AstroDrizzle}\xspace}$
$\newcommand{\dpac}{\textsc{DrizzlePac}\xspace}$
$\newcommand{\fast}{\textsc{FAST}\xspace}$
$\newcommand{\galfit}{\textsc{Galfit}\xspace}$
$\newcommand{\axe}{\textsc{aXe}\xspace}$
$\newcommand{\lt}{\textsc{Lenstool}\xspace}$
$\newcommand{\glafic}{\textsc{Glafic}\xspace}$
$\newcommand{\gasoline}{\textsc{Gasoline}\xspace}$
$\newcommand{\ramses}{\textsc{Ramses}\xspace}$
$\newcommand{\SJ}{\textsc{Sharon \& Johnson}\xspace}$
$\newcommand{\grzl}{\textsc{Grizli}\xspace}$
$\newcommand{\burst}{\textsc{Starburst99}\xspace}$
$\newcommand{\tphot}{\textsc{T-PHOT}\xspace}$
$\newcommand{\bagp}{\textsc{BAGPIPES}\xspace}$
$\newcommand{\phut}{\textsc{Photutils}\xspace}$
$\newcommand{\msa}{\textsc{msaexp}\xspace}$
$\newcommand{\ppxf}{\textsc{pPXF}\xspace}$
$\newcommand{\jwst}{\textit{JWST}\xspace}$
$\newcommand{\planck}{\textit{Planck}\xspace}$
$\newcommand{\hst}{\textit{HST}\xspace}$
$\newcommand{\hubble}{\textit{Hubble}\xspace}$
$\newcommand{\spitzer}{\textit{Spitzer}\xspace}$
$\newcommand{\herschel}{\textit{Herschel}\xspace}$
$\newcommand{\chandra}{\textit{Chandra}\xspace}$
$\newcommand{\glass}{\textit{GLASS}\xspace}$
$\newcommand{\clear}{\textit{CLEAR}\xspace}$
$\newcommand{\wisp}{\textit{WISP}\xspace}$
$\newcommand{\clash}{\textit{CLASH}\xspace}$
$\newcommand{\candels}{\textit{CANDELS}\xspace}$
$\newcommand{\uvc}{\textit{UVCANDELS}\xspace}$
$\newcommand{\hff}{\textit{HFF}\xspace}$
$\newcommand{\muse}{\textit{MUSE}\xspace}$
$\newcommand{\kmos}{\textit{KMOS}\xspace}$
$\newcommand{\keck}{\textit{Keck}\xspace}$
$\newcommand{\deimos}{\textit{DEIMOS}\xspace}$
$\newcommand{\mosfire}{\textit{MOSFIRE}\xspace}$
$\newcommand{\surfsup}{\textit{SURFSUP}\xspace}$
$\newcommand{\kd}{\textit{KMOS}^{3\rm D}\xspace}$
$\newcommand{\sdss}{\textit{SDSS}\xspace}$
$\newcommand{\clash}{\textit{CLASH}\xspace}$
$\newcommand{\mosnewcommand}{\textit{MOSDEF}\xspace}$
$\newcommand{\vlt}{\textit{VLT}\xspace}$
$\newcommand{\osiris}{\textit{OSIRIS}\xspace}$
$\newcommand{\sinf}{\textit{SINFONI}\xspace}$
$\newcommand{\ngrst}{\textit{NGRST}\xspace}$
$\newcommand{\niriss}{\textit{NIRISS}\xspace}$
$\newcommand{\mmth}{\textit{MAMMOTH}\xspace}$
$\newcommand{\mg}{\textit{MAMMOTH-Grism}\xspace}$
$\newcommand{\etal}{et al.\xspace}$
$\newcommand{\ie}{i.e.\xspace}$
$\newcommand{\eg}{e.g.\xspace}$
$\newcommand{\etc}{etc.\xspace}$
$\newcommand{\aka}{a.k.a.\xspace}$
$\newcommand{\vsv}{vis-á-vis\xspace}$
$\newcommand\({\left(}$
$\newcommand\){\right)}$
$\newcommand\sfr{star-formation rate\xspace}$
$\newcommand\sfh{star-formation history\xspace}$
$\newcommand\sfms{star-formation main sequence\xspace}$
$\newcommand{\el}[1]{\ensuremath{\textrm{EL}_{#1}}}$
$\newcommand{\obs}{\textrm{o}}$
$\newcommand{\theo}{\textrm{t}}$
$\newcommand{\ext}{\textrm{ext}}$
$\newcommand{\det}{\textrm{det}}$
$\newcommand\refe{\textrm{ref}}$
$\newcommand\pa{\textrm{PA}}$
$\newcommand{\p}{{\rm prior}}$
$\newcommand{\fid}{{\rm fid}}$
$\newcommand{\lnk}{\kappa}$
$\newcommand{\lnkp}{\kappa'}$
$\newcommand{\xa}{\alpha}$
$\newcommand{\xb}{\beta}$
$\newcommand{\xk}{\kappa}$
$\newcommand{\xg}{\gamma}$
$\newcommand{\Or}{\ensuremath{\Omega_{\rm{r}}}\xspace}$
$\newcommand{\Om}{\ensuremath{\Omega_{\rm{m}}}\xspace}$
$\newcommand{\Ok}{\ensuremath{\Omega_{\rm{k}}}\xspace}$
$\newcommand{\Ol}{\ensuremath{\Omega_{\Lambda}}\xspace}$
$\newcommand{\Obh}{\ensuremath{\Omega_{\rm{b}}h^2}\xspace}$
$\newcommand{\Ob}{\ensuremath{\Omega_{\rm{b}}}\xspace}$
$\newcommand{\Onu}{\ensuremath{\Omega_\nu}\xspace}$
$\newcommand{\fnu}{\ensuremath{f_{\nu}}\xspace}$
$\newcommand{\Och}{\ensuremath{\Omega_{\rm{DM}}h^2}\xspace}$
$\newcommand{\Oc}{\ensuremath{\Omega_{\rm{DM}}}\xspace}$
$\newcommand{\ns}{\ensuremath{n_{\rm s}}\xspace}$
$\newcommand{\As}{\ensuremath{A_{\rm s}}\xspace}$
$\newcommand{\thA}{\ensuremath{\theta_{\rm A}}\xspace}$
$\newcommand{\neff}{\ensuremath{N_\textrm{eff}}\xspace}$
$\newcommand{\mnu}{\ensuremath{\sum{m_{\nu}}}\xspace}$
$\newcommand{\yhe}{\ensuremath{Y_p}\xspace}$
$\newcommand{\Map}[1]{\left<M^2_\textrm{ap}\right>( #1 )}$
$\newcommand{\map}{\ensuremath{\left<M^2_\textrm{ap}\right>}\xspace}$
$\newcommand{\chiH}{\ensuremath{\chi_\textrm{H}}\xspace}$
$\newcommand{\n}{\ensuremath{{\nu}\rm}\xspace}$
$\newcommand{\nue}{\ensuremath{{\nu}_{\rm e}}\xspace}$
$\newcommand{\num}{\ensuremath{{\nu}_{\rm \mu}}\xspace}$
$\newcommand{\nut}{\ensuremath{{\nu}_{\rm \tau}}\xspace}$
$\newcommand{\da}{\ensuremath{D_{\rm A}}\xspace}$
$\newcommand{\dl}{\ensuremath{D_{\rm L}}\xspace}$
$\newcommand{\taueq}{\ensuremath{\tau_{\rm eq}}\xspace}$
$\newcommand{\hi}{H \textsc{i}}$
$\newcommand{\heii}{He \textsc{ii}}$
$\newcommand{\civ}{C \textsc{iv}}$
$\newcommand{\ciii}{[C \textsc{iii}]}$
$\newcommand{\oii}{O \textsc{ii}}$
$\newcommand{\oiii}{O \textsc{iii}}$
$\newcommand{\ovi}{O \textsc{vi}}$
$\newcommand{\Ha}{\textrm{H}\ensuremath{\alpha}\xspace}$
$\newcommand{\Hb}{\textrm{H}\ensuremath{\beta}\xspace}$
$\newcommand{\Hg}{\textrm{H}\ensuremath{\gamma}\xspace}$
$\newcommand{\Hd}{\textrm{H}\ensuremath{\delta}\xspace}$
$\newcommand{\OII}{[\textrm{O}~\textsc{ii}]\xspace}$
$\newcommand{\OIII}{[\textrm{O}~\textsc{iii}]\xspace}$
$\newcommand{\NII}{[\textrm{N}~\textsc{ii}]\xspace}$
$\newcommand{\red}{\color{red}}$
$\newcommand{\blue}{\color{blue}}$
$\newcommand{\logm}{\log(M_*/M_\odot)}$
$\usepackage{graphicx}$
$\usepackage{xspace}$
$\usepackage{subcaption}$
$\usepackage{amsmath}$
$\usepackage{multirow}$
$\usepackage{pgffor}$
$\newcommand{\oh}{\ensuremath{12+\log({\rm O/H})}\xspace}$
$\newcommand{\el}[1]{\ensuremath{\textrm{EL}_{#1}}}$
$\newcommand{\Msun}{\ensuremath{M_\odot}\xspace}$
$\newcommandcitealias{Erb_08}{Erb08}$
$\graphicspath{{./}{img/}}$
$\usepackage[flushleft]{threeparttable}$
$\begin{document}$
$\title{A 13-Billion-Year View of Galaxy Growth: Metallicity Gradient Evolution from the Local Universe to z=9 with JWST and Archival Surveys}$
$\author[0000-0001-5951-459X]{Zihao Li}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\affiliation{Cosmic Dawn Center (DAWN), Denmark}$
$\affiliation{Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Copenhagen N, Denmark}$
$\email[show]{zihao.li@nbi.ku.dk}$
$\author[0000-0001-8467-6478]{Zheng Cai}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\email[show]{zcai@tsinghua.edu.cn}$
$\author[0000-0002-9373-3865]{Xin Wang}$
$\affiliation{School of Astronomy and Space Science, University of Chinese Academy of Sciences, Beijing 100049, China}$
$\affiliation{National Astronomical Observatories, Chinese Academy of Sciences, Beijing 100101, China}$
$\affiliation{Institute for Frontiers in Astronomy and Astrophysics, Beijing Normal University, Beijing 102206, China}$
$\email[show]{xwang@ucas.ac.cn}$
$\author[0000-0001-7890-4964]{Zhaozhou Li}$
$\affiliation{Center for Astrophysics and Planetary Science, Racah Institute of Physics, The Hebrew University, Jerusalem, 91904, Israel}$
$\email $
$\author[0000-0003-4174-0374]{Avishai Dekel}$
$\affiliation{Center for Astrophysics and Planetary Science, Racah Institute of Physics, The Hebrew University, Jerusalem, 91904, Israel}$
$\affiliation{Santa Cruz Institute for Particle Physics, University of California, Santa Cruz, CA 95064, USA}$
$\email $
$\author[0000-0002-7767-8472]{Kartick C. Sarkar}$
$\affiliation{Center for Astrophysics and Planetary Science, Racah Institute of Physics, The Hebrew University, Jerusalem, 91904, Israel}$
$\affiliation{Raman Research Institute, Sadashivanagar, C. V. Raman Avenue, 560 080, Bangalore, India}$
$\email $
$\author[0000-0002-2931-7824]{Eduardo Bañados}$
$\affiliation{Max-Plank-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\email $
$\author[0000-0002-1620-0897]{Fuyan Bian}$
$\affiliation{European Southern Observatory, Alonso de Cordova 3107, Casilla 19001, Vitacura, Santiago 19, Chile}$
$\email $
$\author[0000-0002-7080-2864]{Aklant K. Bhowmick}$
$\affiliation{Department of Physics, University of Florida, Gainesville, Florida 32601, USA}$
$\email $
$\author[0000-0002-2183-1087]{Laura Blecha}$
$\affiliation{Department of Physics, University of Florida, Gainesville, Florida 32601, USA}$
$\email $
$\author[0000-0001-8582-7012]{Sarah E. I. Bosman}$
$\affiliation{Max-Plank-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\affiliation{Institute for Theoretical Physics, Heidelberg University, Philosophenweg 12, D–69120, Heidelberg, Germany}$
$\email $
$\author[0000-0002-6184-9097]{Jaclyn B. Champagne}$
$\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Avenue, Tucson, AZ 85721, USA}$
$\email $
$\author[0000-0002-6336-3007]{Xiaohui Fan}$
$\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Avenue, Tucson, AZ 85721, USA}$
$\email $
$\author[0000-0001-5160-6713]{Emmet Golden-Marx}$
$\affiliation{INAF-Astronomical Observatory of Padova vicolo dell’Osservatorio 5 35122 Padova, Italy}$
$\email $
$\author[0000-0003-1470-5901]{Hyunsung D. Jun}$
$\affiliation{Department of Physics, Northwestern College, 101 7th Street SW, Orange City, Iowa 51041, USA}$
$\email $
$\author[0000-0001-6251-649X]{Mingyu Li}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\email $
$\author[0000-0001-6052-4234]{Xiaojing Lin}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Avenue, Tucson, AZ 85721, USA}$
$\email $
$\author[0000-0003-3762-7344]{Weizhe Liu}$
$\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Avenue, Tucson, AZ 85721, USA}$
$\email $
$\author[0000-0002-4622-6617]{Fengwu Sun}$
$\affiliation{Center for Astrophysics | Harvard \& Smithsonian, 60 Garden St., Cambridge, MA 02138, USA}$
$\email $
$\author[0000-0002-6849-5375]{Maxime Trebitsch}$
$\affiliation{LUX, Observatoire de Paris, Université PSL, Sorbonne Université, CNRS, 75014 Paris, France}$
$\email $
$\author[0000-0003-4793-7880]{Fabian Walter}$
$\affiliation{Max-Plank-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\email $
$\author[0000-0002-7633-431X]{Feige Wang}$
$\affiliation{Department of Astronomy, University of Michigan, 500 S State St, Ann Arbor, MI 48109, USA}$
$\email $
$\author[0000-0003-0111-8249]{Yunjing Wu}$
$\affiliation{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\email $
$\author[0000-0001-5287-4242]{Jinyi Yang}$
$\affiliation{Department of Astronomy, University of Michigan, 500 S State St, Ann Arbor, MI 48109, USA}$
$\email $
$\author[0000-0002-0123-9246]{Huanian Zhang}$
$\affiliation{Department of Astronomy, Huazhong University of Science and Technology, Wuhan 430074, China}$
$\email $
$\author[0000-0002-0427-9577]{Shiwu Zhang}$
$\affiliation{Zhejiang Lab, Hangzhou, Zhejiang 311121, China}$
$\email $
$\author[0000-0001-5105-2837]{Mingyang Zhuang}$
$\affiliation{Department of Astronomy, University of Illinois at Urbana-Champaign, Urbana, IL 61801, USA}$
$\email $
$\author[0000-0002-3983-6484]{Siwei Zou}$
$\affiliation{Chinese Academy of Sciences South America Center for Astronomy, National Astronomical Observatories, CAS, Beijing 100101, China}$
$\affiliation{Departamento de Astronomía, Universidad de Chile, Casilla 36-D, Santiago, Chile}$
$\email $
$\begin{abstract}$
$The gas-phase metallicity gradients of galaxies have been extensively studied over the past four decades both in the local and high-redshift universe, as they trace the baryon cycle and growth of galaxies. With the unprecedented spatial resolution and sensitivity of the James Webb Space Telescope (JWST), it is now possible to measure metallicity and its radial gradients out to redshifts as high as z = 9. Here, we present a sample of 456 spectroscopically confirmed galaxies from redshifts 1.5 \lesssim z \lesssim 9 that are spatially resolved on sub-kiloparsec (kpc) scales by deep JWST NIRCam or NIRISS Wide Field Slitless Spectroscopy (WFSS). Synthesizing these new JWST observations with legacy observations from the literature, we observe that at redshift z > 5, galaxy centers are more metal-rich, exhibiting negative metallicity gradients of \sim-0.4 dex kpc^{-1}. These gradients flatten over time, reaching near-zero around z \approx 2, coinciding with the peak of the cosmic star formation rate. Beyond this point, the gradients become negative again at lower redshifts approaching z=0.$
$This evolution likely reflects transitions in galaxy formation modes: an inside-out growth phase dominated by intense central star formation with inefficient feedback and limited gas mixing during "cosmic dawn", enhanced gas mixing due to feedback-driven wind and gas accretion at "cosmic noon", and a later phase of slow evolution and reduced feedback toward the present day. These physical processes including gas accretion and feedback not only regulate star and galaxy formation on cosmic scale, but also shape the evolutionary pathways of individual galaxies over cosmic time.$
$\end{abstract}$
$\keywords{\uat{Galaxies}{573} --- \uat{High-redshift galaxies}{734} --- \uat{Chemical enrichment}{225} --- \uat{Galaxy evolution}{594} --- \uat{Galaxy formation}{595} --- \uat{Metallicity}{1031}}$
$\n\end{document}\end{align}}$
$\newcommand{\ea}{\end{align}}$
$\newcommand{\nt}{\notag}$
$\newcommand{\avg}[1]{\left<#1\right>}$
$\newcommand{\newcommandeq}{\vcentcolon=}$
$\newcommand{\eqnewcommand}{=\vcentcolon}$
$\newcommand{\Ra}{\ensuremath{\Rightarrow}\xspace}$
$\newcommand{\ra}{\ensuremath{\rightarrow}\xpace}$
$\newcommand{\lra}{\ensuremath{\Leftrightarrow}\xspace}$
$\newcommand{\stmean}[1]{\langle{#1}\rangle}$
$\newcommand{\logg}{\log_{10}}$
$\newcommand{\Msun}{\ensuremath{M_\odot}\xspace}$
$\newcommand{\Rsun}{\ensuremath{R_\odot}\xspace}$
$\newcommand{\Lsun}{\ensuremath{L_\odot}\xspace}$
$\newcommand{\thE}{\ensuremath{\theta_{\rm E}}\xspace}$
$\newcommand{\chisq}{\ensuremath{\chi^2}\xspace}$
$\newcommand{\zspec}{\ensuremath{z_{\rm spec}}\xspace}$
$\newcommand{\zphot}{\ensuremath{z_{\rm phot}}\xspace}$
$\newcommand{\Mstar}{\ensuremath{M_\ast}\xspace}$
$\newcommand{\Lstar}{\ensuremath{L_\ast}\xspace}$
$\newcommand{\Sstar}{\ensuremath{\Sigma_\ast}\xspace}$
$\newcommand{\oh}{\ensuremath{12+\log({\rm O/H})}\xspace}$
$\newcommand{\Av}{\ensuremath{A_{\rm V}}\xspace}$
$\newcommand{\Rv}{\ensuremath{R_{\rm V}}\xspace}$
$\newcommand{\Te}{\ensuremath{T_{\rm e}}\xspace}$
$\newcommand{\SFR}{\ensuremath{{\rm SFR}}\xspace}$
$\newcommand{\Mgas}{\ensuremath{M_{\rm gas}}\xspace}$
$\newcommand{\Sgas}{\ensuremath{\Sigma_{\rm gas}}\xspace}$
$\newcommand{\fgas}{\ensuremath{f_{\rm gas}}\xspace}$
$\newcommand{\Zgas}{\ensuremath{Z_{\rm gas}}\xspace}$
$\newcommand{\tage}{\ensuremath{t_{\rm age}}\xspace}$
$\newcommand{\Vrot}{\ensuremath{V_{\rm rot}}\xspace}$
$\newcommand{\reff}{\ensuremath{r_{\rm eff}}\xspace}$
$\newcommand{\Dn}{\ensuremath{{\rm D}_n(4000)}\xspace}$
$\newcommand{\HdA}{\ensuremath{{\rm H}\delta_A}\xspace}$
$\newcommand{\scrit}{\ensuremath{\sigma_{\rm crit}}\xspace}$
$\newcommand{\fesc}{\ensuremath{f_{\rm esc}}\xspace}$
$\newcommand{\xion}{\ensuremath{\xi_{\rm ion}}\xspace}$
$\newcommand{\tauIGM}{\ensuremath{\tau_{\rm IGM}}\xspace}$
$\newcommand{\Muv}{\ensuremath{M_{\rm UV}}\xspace}$
$\newcommand{\eV}{\ensuremath{\rm eV}\xspace}$
$\newcommand{\pc}{\ensuremath{\rm pc}\xspace}$
$\newcommand{\kpc}{\ensuremath{\rm kpc}\xspace}$
$\newcommand{\Mpc}{\ensuremath{\rm Mpc}\xspace}$
$\newcommand{\K}{\ensuremath{\rm K}\xspace}$
$\newcommand{\mK}{\ensuremath{\rm mK}\xspace}$
$\newcommand{\Hunit}{\ensuremath{\rm km~s^{-1}~Mpc^{-1}}\xspace}$
$\newcommand{\Funit}{\ensuremath{\rm erg~s^{-1}~cm^{-2}}\xspace}$
$\newcommand{\Flam}{\ensuremath{\rm erg~s^{-1}~cm^{-2}~Å^{-1}}\xspace}$
$\newcommand{\Fnu}{\ensuremath{\rm erg~s^{-1}~cm^{-2}~Hz^{-1}}\xspace}$
$\newcommand{\muJy}{\ensuremath{\mu\rm Jy}\xspace}$
$\newcommand{\SBunit}{\ensuremath{\rm erg~s^{-1}~cm^{-2}~arcsec^{-2}}\xspace}$
$\newcommand{\magarcs}{\ensuremath{\rm mag~arcsec^{-2}}\xspace}$
$\newcommand{\Msunyr}{\ensuremath{\Msun~\mathrm{yr}^{-1}}\xspace}$
$\newcommand{\yr}{\ensuremath{\rm yr}\xspace}$
$\newcommand{\Myr}{\ensuremath{\rm Myr}\xspace}$
$\newcommand{\Gyr}{\ensuremath{\rm Gyr}\xspace}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\Ha}{\textrm{H}\ensuremath{\alpha}\xspace}$
$\newcommand{\Hb}{\textrm{H}\ensuremath{\beta}\xspace}$
$\newcommand{\Hg}{\textrm{H}\ensuremath{\gamma}\xspace}$
$\newcommand{\Hd}{\textrm{H}\ensuremath{\delta}\xspace}$
$\newcommand{\HII}{\textrm{H}\textsc{ii}\xspace}$
$\newcommand{\HI}{\textrm{H}\textsc{i}\xspace}$
$\newcommand{\Htwo}{\textrm{H}\ensuremath{_2}\xspace}$
$\newcommand{\He}{\textrm{He}\xspace}$
$\newcommand{\OI}{[\textrm{O}~\textsc{i}]\xspace}$
$\newcommand{\OII}{[\textrm{O}~\textsc{ii}]\xspace}$
$\newcommand{\OIII}{[\textrm{O}~\textsc{iii}]\xspace}$
$\newcommand{\CIII}{\textrm{C}~\textsc{iii}]\xspace}$
$\newcommand{\NII}{[\textrm{N}~\textsc{ii}]\xspace}$
$\newcommand{\SII}{[\textrm{S}~\textsc{ii}]\xspace}$
$\newcommand{\NeIII}{[\textrm{Ne}~\textsc{iii}]\xspace}$
$\newcommand{\HeII}{\textrm{He}~\textsc{ii}\xspace}$
$\newcommand{\CIV}{\textrm{C}~\textsc{iv}\xspace}$
$\newcommand{\SiIV}{\textrm{Si}~\textsc{iv}\xspace}$
$\newcommand{\CII}{\textrm{C}~\textsc{ii}\xspace}$
$\newcommand{\sersic}{Sérsic\xspace}$
$\newcommand{\clyi}{MACS1149.6+2223\xspace}$
$\newcommand{\cler}{Abell 2744\xspace}$
$\newcommand{\clsan}{Abell 370\xspace}$
$\newcommand{\clsi}{MACS0416.1-2403\xspace}$
$\newcommand{\clwu}{MACS0717.5+3745\xspace}$
$\newcommand{\clliu}{RXJ2248.7-4431\xspace}$
$\newcommand{\clqi}{RXJ1347.5-1145\xspace}$
$\newcommand{\clba}{MACS0744.9+3927\xspace}$
$\newcommand{\cljiu}{MACS2129.4-0741\xspace}$
$\newcommand{\clshi}{MACS1423.8+2404\xspace}$
$\newcommand{\pylf}{\textsc{pyLensFix}\xspace}$
$\newcommand{\lf}{\textsc{LensFix}\xspace}$
$\newcommand{\sw}{\textsc{SWunited}\xspace}$
$\newcommand{\sex}{\textsc{SExtractor}\xspace}$
$\newcommand{\emc}{\textsc{Emcee}\xspace}$
$\newcommand{\linmix}{\textsc{linmix}\xspace}$
$\newcommand{\adriz}{\textsc{AstroDrizzle}\xspace}$
$\newcommand{\dpac}{\textsc{DrizzlePac}\xspace}$
$\newcommand{\fast}{\textsc{FAST}\xspace}$
$\newcommand{\galfit}{\textsc{Galfit}\xspace}$
$\newcommand{\axe}{\textsc{aXe}\xspace}$
$\newcommand{\glafic}{\textsc{Glafic}\xspace}$
$\newcommand{\gasoline}{\textsc{Gasoline}\xspace}$
$\newcommand{\ramses}{\textsc{Ramses}\xspace}$
$\newcommand{\SJ}{\textsc{Sharon \& Johnson}\xspace}$
$\newcommand{\grzl}{\textsc{Grizli}\xspace}$
$\newcommand{\burst}{\textsc{Starburst99}\xspace}$
$\newcommand{\tphot}{\textsc{T-PHOT}\xspace}$
$\newcommand{\bagp}{\textsc{BAGPIPES}\xspace}$
$\newcommand{\phut}{\textsc{Photutils}\xspace}$
$\newcommand{\msa}{\textsc{msaexp}\xspace}$
$\newcommand{\ppxf}{\textsc{pPXF}\xspace}$
$\newcommand{\jwst}{\textit{JWST}\xspace}$
$\newcommand{\planck}{\textit{Planck}\xspace}$
$\newcommand{\hst}{\textit{HST}\xspace}$
$\newcommand{\hubble}{\textit{Hubble}\xspace}$
$\newcommand{\spitzer}{\textit{Spitzer}\xspace}$
$\newcommand{\herschel}{\textit{Herschel}\xspace}$
$\newcommand{\chandra}{\textit{Chandra}\xspace}$
$\newcommand{\glass}{\textit{GLASS}\xspace}$
$\newcommand{\clear}{\textit{CLEAR}\xspace}$
$\newcommand{\wisp}{\textit{WISP}\xspace}$
$\newcommand{\clash}{\textit{CLASH}\xspace}$
$\newcommand{\candels}{\textit{CANDELS}\xspace}$
$\newcommand{\uvc}{\textit{UVCANDELS}\xspace}$
$\newcommand{\hff}{\textit{HFF}\xspace}$
$\newcommand{\muse}{\textit{MUSE}\xspace}$
$\newcommand{\kmos}{\textit{KMOS}\xspace}$
$\newcommand{\keck}{\textit{Keck}\xspace}$
$\newcommand{\deimos}{\textit{DEIMOS}\xspace}$
$\newcommand{\mosfire}{\textit{MOSFIRE}\xspace}$
$\newcommand{\surfsup}{\textit{SURFSUP}\xspace}$
$\newcommand{\kd}{\textit{KMOS}^{3\rm D}\xspace}$
$\newcommand{\sdss}{\textit{SDSS}\xspace}$
$\newcommand{\vlt}{\textit{VLT}\xspace}$
$\newcommand{\osiris}{\textit{OSIRIS}\xspace}$
$\newcommand{\sinf}{\textit{SINFONI}\xspace}$
$\newcommand{\ngrst}{\textit{NGRST}\xspace}$
$\newcommand{\niriss}{\textit{NIRISS}\xspace}$
$\newcommand{\mmth}{\textit{MAMMOTH}\xspace}$
$\newcommand{\mg}{\textit{MAMMOTH-Grism}\xspace}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\el}[1]{\ensuremath{\textrm{EL}_{#1}}}$
$\newcommand{\obs}{\textrm{o}}$
$\newcommand{\theo}{\textrm{t}}$
$\newcommand{\ext}{\textrm{ext}}$
$\newcommand$
$\newcommand$
$\newcommand{\xa}{\alpha}$
$\newcommand{\xb}{\beta}$
$\newcommand{\xk}{\kappa}$
$\newcommand{\xg}{\gamma}$
$\newcommand{\Or}{\ensuremath{\Omega_{\rm{r}}}\xspace}$
$\newcommand{\Om}{\ensuremath{\Omega_{\rm{m}}}\xspace}$
$\newcommand{\Ok}{\ensuremath{\Omega_{\rm{k}}}\xspace}$
$\newcommand{\Ol}{\ensuremath{\Omega_{\Lambda}}\xspace}$
$\newcommand{\Obh}{\ensuremath{\Omega_{\rm{b}}h^2}\xspace}$
$\newcommand{\Ob}{\ensuremath{\Omega_{\rm{b}}}\xspace}$
$\newcommand{\Onu}{\ensuremath{\Omega_\nu}\xspace}$
$\newcommand{\fnu}{\ensuremath{f_{\nu}}\xspace}$
$\newcommand{\Och}{\ensuremath{\Omega_{\rm{DM}}h^2}\xspace}$
$\newcommand{\Oc}{\ensuremath{\Omega_{\rm{DM}}}\xspace}$
$\newcommand{\ns}{\ensuremath{n_{\rm s}}\xspace}$
$\newcommand{\As}{\ensuremath{A_{\rm s}}\xspace}$
$\newcommand{\thA}{\ensuremath{\theta_{\rm A}}\xspace}$
$\newcommand{\neff}{\ensuremath{N_\textrm{eff}}\xspace}$
$\newcommand{\mnu}{\ensuremath{\sum{m_{\nu}}}\xspace}$
$\newcommand{\yhe}{\ensuremath{Y_p}\xspace}$
$\newcommand{\Map}[1]{\left<M^2_\textrm{ap}\right>( #1 )}$
$\newcommand{\map}{\ensuremath{\left<M^2_\textrm{ap}\right>}\xspace}$
$\newcommand{\chiH}{\ensuremath{\chi_\textrm{H}}\xspace}$
$\newcommand{\n}{\ensuremath{{\nu}\rm}\xspace}$
$\newcommand{\nue}{\ensuremath{{\nu}_{\rm e}}\xspace}$
$\newcommand{\num}{\ensuremath{{\nu}_{\rm \mu}}\xspace}$
$\newcommand{\nut}{\ensuremath{{\nu}_{\rm \tau}}\xspace}$
$\newcommand{\da}{\ensuremath{D_{\rm A}}\xspace}$
$\newcommand{\dl}{\ensuremath{D_{\rm L}}\xspace}$
$\newcommand{\taueq}{\ensuremath{\tau_{\rm eq}}\xspace}$
$\newcommand{\hi}{H \textsc{i}}$
$\newcommand{\heii}{He \textsc{ii}}$
$\newcommand{\civ}{C \textsc{iv}}$
$\newcommand{\ciii}{[C \textsc{iii}]}$
$\newcommand{\oii}{O \textsc{ii}}$
$\newcommand{\oiii}{O \textsc{iii}}$
$\newcommand{\ovi}{O \textsc{vi}}$
$\newcommand{\Ha}{\textrm{H}\ensuremath{\alpha}\xspace}$
$\newcommand{\Hb}{\textrm{H}\ensuremath{\beta}\xspace}$
$\newcommand{\Hg}{\textrm{H}\ensuremath{\gamma}\xspace}$
$\newcommand{\Hd}{\textrm{H}\ensuremath{\delta}\xspace}$
$\newcommand{\OII}{[\textrm{O}~\textsc{ii}]\xspace}$
$\newcommand{\OIII}{[\textrm{O}~\textsc{iii}]\xspace}$
$\newcommand{\NII}{[\textrm{N}~\textsc{ii}]\xspace}$
$\newcommand{\red}{\color{red}}$
$\newcommand{\blue}{\color{blue}}$
$\newcommand{\logm}{\log(M_*/M_\odot)}$
$\newcommand{\oh}{\ensuremath{12+\log({\rm O/H})}\xspace}$
$\newcommand{\el}[1]{\ensuremath{\textrm{EL}_{#1}}}$
$\newcommand{\Msun}{\ensuremath{M_\odot}\xspace}$
$\newcommand$
$\newcommand$
$\newcommand{\prt}{\partial}$
$\newcommand{\cP}{{\cal P}}$
$\newcommand\abs{#1}$
$\newcommand{\ne}{\ensuremath{n_{\rm e}}\xspace}$
$\newcommand{\micron}{\ensuremath{\mu\textrm{m}}\xspace}$
$\newcommand{\U}{\ensuremath{U_{360}}\xspace}$
$\newcommand{\B}{\ensuremath{B_{435}}\xspace}$
$\newcommand{\V}{\ensuremath{V_{606}}\xspace}$
$\newcommand{\I}{\ensuremath{I_{814}}\xspace}$
$\newcommand{\Y}{\ensuremath{Y_{105}}\xspace}$
$\newcommand{\J}{\ensuremath{J_{125}}\xspace}$
$\newcommand{\JH}{\ensuremath{JH_{140}}\xspace}$
$\newcommand{\H}{\ensuremath{H_{160}}\xspace}$
$\newcommand{\lt}{\textsc{Lenstool}\xspace}$
$\newcommand{\clash}{\textit{CLASH}\xspace}$
$\newcommand{\mosnewcommand}{\textit{MOSDEF}\xspace}$
$\newcommand{\etal}{et al.\xspace}$
$\newcommand{\ie}{i.e.\xspace}$
$\newcommand{\eg}{e.g.\xspace}$
$\newcommand{\etc}{etc.\xspace}$
$\newcommand{\aka}{a.k.a.\xspace}$
$\newcommand{\vsv}{vis-á-vis\xspace}$
$\newcommand{\det}{\textrm{det}}$
$\newcommand{\p}{{\rm prior}}$
$\newcommand{\fid}{{\rm fid}}$
$\newcommand{\lnk}{\kappa}$
$\newcommand{\lnkp}{\kappa'}$</div>



<div id="title">

# A 13-Billion-Year View of Galaxy Growth: Metallicity Gradient Evolution from the Local Universe to $z=9$ with JWST and Archival Surveys

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.12129-b31b1b.svg)](https://arxiv.org/abs/2506.12129)<mark>Appeared on: 2025-06-17</mark> -  _The main results are presented in Fig. 8. Comments are welcome_

</div>
<div id="authors">

Z. Li, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** The gas-phase metallicity gradients of galaxies have been extensively studied over the past four decades both in the local and high-redshift universe, as they trace the baryon cycle and growth of galaxies. With the unprecedented spatial resolution and sensitivity of the James Webb Space Telescope (JWST), it is now possible to measure metallicity and its radial gradients out to redshifts as high as $z = 9$ . Here, we present a sample of 456 spectroscopically confirmed galaxies from redshifts $1.5 \lesssim z \lesssim 9$ that are spatially resolved on sub-kiloparsec (kpc) scales by deep JWST NIRCam or NIRISS Wide Field Slitless Spectroscopy (WFSS). Synthesizing these new JWST observations with legacy observations from the literature, we observe that at redshift $z > 5$ , galaxy centers are more metal-rich, exhibiting negative metallicity gradients of $\sim-0.4$ dex kpc $^{-1}$ . These gradients flatten over time, reaching near-zero around $z \approx 2$ , coinciding with the peak of the cosmic star formation rate. Beyond this point, the gradients become negative again at lower redshifts approaching $z=0$ .This evolution likely reflects transitions in galaxy formation modes: an inside-out growth phase dominated by intense central star formation with inefficient feedback and limited gas mixing during "cosmic dawn", enhanced gas mixing due to feedback-driven wind and gas accretion at "cosmic noon", and a later phase of slow evolution and reduced feedback toward the present day. These physical processes including gas accretion and feedback not only regulate star and galaxy formation on cosmic scale, but also shape the evolutionary pathways of individual galaxies over cosmic time.

</div>

<div id="div_fig1">

<img src="tmp_2506.12129/img/zgrad-z-obs.png" alt="Fig5" width="100%"/>

**Figure 5. -** The redshift evolution of metallicity gradients. The thin red diamonds show the weighted mean in redshift bins at $z=[0,3.5]$ from both this work and the literature. Red "$\times$" and "+" represent the median stacks of the ASPIRE sample in $z\approx 6$ and FRESCO sample in $z\approx 7$.
 The NGDEEP sample at $z\approx 1-3$ is denoted by pentagons, color-coded by sSFR. Gray contours encompass individual measurements at $z<3$ in the literature using HST  ([Wang, Jones and Treu 2017](), [Wang, Jones and Treu 2020](), [Simons, Papovich and Momcheva 2021]()) , and ground-based AO-assisted surveys  ([Swinbank, Sobral and Smail 2012](), [Jones, et. al 2013](), [Renzini and Mancini 2018]()) . We also include seeing-limited observations  ([Salzer, Haynes and Balonek 1998](), [Queyrel, Contini and Kissler-Patig 2012](), [Curti, Maiolino and Cirasuolo 2020](), [Carton, Brinchmann and Contini 2018](), [Wuyts, Wisnioski and Fossati 2016]()) .
 The red contour only encompasses our observations at $1.7<z<3.5$, overlapping with literature observations at $z\sim2$. Gray circles with error bars represent interacting galaxies at $z>6$ recently observed with JWST NIRSpec  ([Venturi, Carniani and Parlanti 2024]()) . Recent JWST MSA-3D observations  ([Ju, Wang and Jones 2025]())  are noted in circles color-coded by sSFR.
 The dark-red line shows our best fitting of all stacks, with dashed lines in the same color showing the $1\sigma$ confidence interval of the fit.
 In the upper panel, we plot the cosmic star formation rate density (SFRD  ([Madau and Dickinson 2014]()) ), where the evolution of metallicity gradients also peaks at $z\sim2$.
  (*fig:zgrad-z*)

</div>
<div id="div_fig2">

<img src="tmp_2506.12129/./img/ASPIRE_stacking.png" alt="Fig4.1" width="50%"/><img src="tmp_2506.12129/./img/FRESCO_stacking.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Stacking results of $5<z<7$ galaxies in ASPIRE (top) and $7<z<9$ galaxies in FRESCO (bottom). The first two columns show the median stack of emission maps ($\OI$II, $\Hb$). The FWHM of PSFs are shown on the bottom left in each line map. The third column shows the surface brightness (SB) profiles of $\OI$II and $\Hb$, normalized by the peak $\Hb$ flux. The vertical dashed line marks the angular resolution (half of PSF FWHM). The fourth column shows the fitted metallicity gradient.  The black error bars include both statistical error and calibration error.
  The red shadow represents the $1\sigma$ confidence interval of the linear regression. Both samples in each redshift bin show steep gradients towards the center.
     (*fig:zgrad-stacking*)

</div>
<div id="div_fig3">

<img src="tmp_2506.12129/./img/zgrad-m-fit.png" alt="Fig7" width="100%"/>

**Figure 7. -** The mass dependence of metallicity gradients. Galaxies in NGDEEP, ASPIRE, and FRESCO are labeled with a pentagon, "$\times$" and "$+$", respectively, color-coded by redshift. The highest mass bin in ASPIRE is marked with a dashed error bar; for this bin, the metallicity gradients were derived using the higher-branch solution. The red and orange shaded areas represent the $1\sigma$ confidence intervals for the linear fit to the stacked data at $z > 5$ and the individual NGDEEP measurements within $1.5 < z < 3.5$, respectively.
 The literature observations  ([Swinbank, Sobral and Smail 2012](), [Jones, et. al 2013](), [Wuyts, Wisnioski and Fossati 2016](), [Wang, Jones and Treu 2017](), [Carton, Brinchmann and Contini 2018](), [Renzini and Mancini 2018](), [Wang, Jones and Treu 2020](), [Curti, Maiolino and Cirasuolo 2020](), [Simons, Papovich and Momcheva 2021](), [Ju, Wang and Jones 2025]())  are included in gray-filled regions with 1, 2 and 3$\sigma$ levels of the density contours of the distribution. Green, yellow, and blue hatched areas represent the $1\sigma$ interval from FIRE  ([Ma, Hopkins and Feldmann 2017]()) , TNG50  ([Hemler, Torrey and Qi 2021]()) , and EAGLE  ([Tissera, Rosas-Guevara and Sillero 2022]())  simulations, respectively. The dashed magenta and dotted orange lines show the median metallicity gradients of EAGLE galaxies with strong feedback (StrongFB) and weak feedback (WeakFB) prescriptions  ([ and Rosas-Guevara 2019]()) .
  (*fig:zgrad-mass*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.12129"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\NRAO}{\affiliation{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}}$
$\newcommand{\UVa}{\affiliation{Department of Astronomy, University of Virginia, Charlottesville, VA 22904}}$</div>



<div id="title">

# The Star Formation in Radio Survey: Adding 90 GHz Data to 3\textendash33 GHz Observations of Star-forming Regions in Nearby Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.12136-b31b1b.svg)](https://arxiv.org/abs/2506.12136)<mark>Appeared on: 2025-06-17</mark> -  _27 pages, 6 figures_

</div>
<div id="authors">

A. Dignan, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ We present 90 GHz continuum imaging of 119 star-forming regions in 30 nearby galaxies observed with MUSTANG-2 on the Robert C. Byrd Green Bank Telescope as part of the Star Formation in Radio Survey.The 90 GHz data were combined with 3, 15, and 33 GHz data taken previously by the Karl G. Jansky Very Large Array to decompose radio spectra on $\approx$ 0.8 kpc scales into their synchrotron, free-free, and thermal dust emission components.This was done using three scenarios:(i) a power law fit from 3 to 33 GHz,(ii) Markov Chain Monte Carlo (MCMC) fitting from 3 to 90 GHz with a thermal dust component, and(iii) MCMC fitting from 3 to 33 GHz without a thermal dust component.For these cases, we find a median thermal (free-free) emission fraction at 33 GHz of (i) $88 \pm 2$ \% with a scatter of 17 \% , (ii) $76\pm 3$ \% with a scatter of 25 \% , and (iii) $84\pm 2$ \% with a scatter of 18 \% .From this we conclude that, on average, free-free, not thermal dust, remains the dominant emission component at 33 GHz. While scenario (ii) yields a thermal fraction that is $\approx$ 10 \% larger than scenario (iii), this difference decreases to $\approx$ 5 \% after AGN are removed. Consequently, star formation rates measured with thermal fractions at 33 GHz are only mildly biased high without 90 GHz data for the spectral decomposition.Furthermore, a power law fit of data from 3 to 33 GHz still provides a reliable estimate of the free-free emission at 33 GHz.

</div>

<div id="div_fig1">

<img src="tmp_2506.12136/./trends.png" alt="Fig6" width="100%"/>

**Figure 6. -** Top panels: distributions of the thermal (free-free) fractions found using the four-parameter model that included 90 GHz data and thermal dust versus galactocentric radius and photometric aperture diameter. Bottom panels: distributions of the thermal fraction ratios for the four-parameter model versus the three-parameter model that excluded 90 GHz data and thermal dust versus galactocentric radius and photometric aperture diameter. We note low thermal fractions for two of the three regions associated with NGC 2146. Sample size: 118 regions (excluding the source likely associated with a supernova, NGC 6946 Enuc. 6a). (*fig:trends*)

</div>
<div id="div_fig2">

<img src="tmp_2506.12136/./spectralindexhist_powerlaw.png" alt="Fig1" width="100%"/>

**Figure 1. -** Spectral index distributions for two-component power law fitting for 3 to 33 and 33 to 90 GHz. There are clear differences in the peaks and distribution widths; the distribution of the spectral indices measured from 3 to 33 GHz is narrower and peaks at $-0.028$, while the distribution of the spectral indices measured from 33 to 90 GHz is more extended and peaks at 0.06. This demonstrates the need for at least one additional power law component in order to adequately characterize the observed radio spectra. (*fig:spectralindexpowerlaw*)

</div>
<div id="div_fig3">

<img src="tmp_2506.12136/./f1_1.png" alt="Fig4" width="100%"/>

**Figure 4. -** Images at each frequency for the extranuclear region NGC 5194 Enuc. 3. The white circles correspond to the apertures used to perform photometry, while the cyan circles represent the annuli used to determine the local background. A corresponding version of this figure for all other sources is available in the online journal as a figure set (120 images). (*fig:photometry*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.12136"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\A}{A_{2}}$
$\newcommand{\tb}{t_{\rm bar}}$
$\newcommand{\ls}{l_{*}}$
$\newcommand{\lgas}{l_{g}}$
$\newcommand{\Msun}{\mbox{M}_{\sun}}$
$\newcommand{\sigz}{\sigma_{z}}$
$\newcommand{\sigrel}{\sigma_{\rm rel}}$
$\newcommand{\vesc}{v_{\rm esc}}$
$\newcommand{\om}{\Omega}$
$\newcommand{\omb}{\Omega_{b}}$
$\newcommand{\kms}{km s^{-1}}$</div>



<div id="title">

# Simulating nearby disc galaxies on the main star formation sequence: II. The gas structure transition in low  and high stellar mass discs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.12923-b31b1b.svg)](https://arxiv.org/abs/2506.12923)<mark>Appeared on: 2025-06-17</mark> -  _17 pages, 14 figures_

</div>
<div id="authors">

P. Verwilghen, et al. -- incl., <mark>J. Neumann</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Recent  hydrodynamical simulations of isolated barred disc galaxies have suggested a structural change in the distribution of the interstellar medium (ISM) around a stellar mass M $_{*}$ of $10^{10}$  $\Msun$ .  In the higher-mass regime (M $_{*} \geq 10^{10}$ $\Msun$ ), we observe the formation of a central gas and stellar disc with a typical size of a few hundred parsecs connected through lanes to the ends of the stellar bar. In the lower-mass regime (M $_{*} < 10^{10}$ $\Msun$ ), such an inner disc is absent and the gas component exhibits a more chaotic distribution. Observations of nearby star-forming galaxies support the existence of such a change. These inner gas discs may represent an important intermediate scale connecting the large kiloparsec-scale structures with the nuclear (sub-parsec) region, transporting gas  inwards to fuel the central supermassive black hole (SMBH). For this work we used an extended set of high-resolution hydrodynamical simulations of isolated disc galaxies with initial properties (i.e. stellar mass, gas fraction, stellar disc scale length, and the bulge mass fraction) with properties covering the range of galaxies in the PHANGS sample to investigate this change of regime. We studied the physical properties of the star-forming ISM in both stellar mass regimes and extracted a few physical tracers: the inner Lindblad resonance (ILR), the probability distribution function (PDF), the virial parameter, and the Mach number. In line with observations, we confirm a structure transition in the simulations that occurs between a stellar mass of $10^{9.5}$ and $10^{10}$ $\Msun$ . We show that the physical origin of this change of regime is driven by stellar feedback and its contribution relative to the underlying gravitational potential. With their shallower potential and typically higher gas mass fraction, lower-mass disc PHANGS galaxies combine two ingredients that significantly delay or even prevent the formation of a central gas (and stellar) disc. These results shed some light on the observed properties of star-forming barred galaxies and have implications for the star formation regimes, the growth of central structures, and the overall secular evolution of disc galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2506.12923/./figures/sigz_vesc_low_high_masses.png" alt="Fig9" width="100%"/>

**Figure 9. -** Evolution (from left to right) of the vertical velocity dispersion $\sigma_z$(two bottom rows) and $\sigma_z$ normalised by the local escape velocity (two top rows). The blue (resp. red) curves represent models with an initial stellar mass of $10^{9.5}$ $\Msun$(resp. above or equal to $10^{10}$ $\Msun$, see inset in  top left panel). The grey lines are models that do not display a bar at the end of the simulation. The line thickness corresponds to the gas fraction (see inset in the top middle  panel). The green shaded areas show vertical velocity dispersions ranging from 10 to 40 $\kms$ and the purple shaded areas show a percentage above 5\%. (*fig:feedback_potential*)

</div>
<div id="div_fig2">

<img src="tmp_2506.12923/./figures/gas_star_density_map_tau.png" alt="Fig6" width="100%"/>

**Figure 6. -** Surface density map of gas and newly formed stars ($\leq 100$ Myr) of four models in two stellar mass bins (G013, G025, with an initial stellar mass of $10^{9.5}$ $\Msun$; G037, and G053 with an initial stellar mass of $10^{10}$ $\Msun$) from our set of simulations. Each panel shows one model at different values of the parameter $\tau$, with $\tau=1$ corresponding to the bar formation timescale. G013 and G053 have the same 20\% gas mass fraction, while G025 and G037 have a 40\% and 10\% gas mass fraction, respectively (the latter two following the averaged properties of star-forming main-sequence galaxies at their respective stellar masses). (*fig:sig_gas_tau05-5*)

</div>
<div id="div_fig3">

<img src="tmp_2506.12923/./figures/pdf_mass_tau1_2_5_lognorm.png" alt="Fig2" width="100%"/>

**Figure 2. -** Evolution of the mass-weighted gas density PDFs as a function of $\tau$(for $\tau=1$, 2, and 5) of four of our simulated galaxies (see also Fig. \ref{fig:sig_gas_tau05-5}), namely models G013, G025 with initial stellar masses of $10^{9.5}$ $\Msun$(top row), and G037 and G053 with initial stellar masses of $10^{10}$ $\Msun$(bottom row). The bottom panels in each row show the PDF normalised by the PDF at $\tau=1$(NPDF). The faded grey curve shows a fit of a Log-normal to the PDF at $\tau=1$ of model G013 and serves as a reference in all panels. (*fig:pdfnorm*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.12923"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\citeg}[1]{\citep[e.g.,][]{#1}}$
$\newcommand{\tabincell}[2]{\begin{tabular}{@ #1@ }#2\end{tabular}}$
$\newcommand{\indentpar}[1]{\noindent\hangindent=2em\hangafter=1 #1}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Stellar initial mass function in the 100-pc solar neighbourhood

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.12987-b31b1b.svg)](https://arxiv.org/abs/2506.12987)<mark>Appeared on: 2025-06-17</mark> -  _Submitted to MNRAS_

</div>
<div id="authors">

Y.-T. Wang, C. Liu, <mark>J. Li</mark>

</div>
<div id="abstract">

**Abstract:** The stellar initial mass function (IMF) is one of the most important astrophysical distribution function, and is defined as the mass distribution for stars formed in a single star-formation event and when they just enter the main-sequence stage. The measurement of IMF is challenging even in the solar neighbourhood where IMF can be extracted by star-counting. This is because the definition of the IMF should be handled cautiously, and there are multiple corrections should be made from observation to a well-founded measurement. We present a new parametrisation of the stellar IMF in the 100-pc solar neighbourhood, leveraging the astrometric and photometric data from \textsl{Gaia} DR3: we model the colour-magnitude diagram of the field star population while accounting for observational uncertainties, Malmquist bias, Lutz-Kelker bias, the effect of varying mass-luminosity relation caused by mixture of different metallicity, and the effect of unresolved binaries. In particular, we synthesise the binaries with a process imitating the dynamical evolution observed in star clusters to model their present-day mass-ratio distribution. We derive a stellar IMF consistent with canonical IMFs but with significantly reduced uncertainties: $\alpha_1=0.81^{+0.06}_{-0.05}$ , $\alpha_2=2.12^{+0.04}_{-0.04}$ , and a break point at $m_{\mathrm{break}}=0.41^{+0.01}_{-0.01}$ $\mathrm{M_{\odot}}$ . Our model also constrains the binary fraction to $\sim$ 25 \% and the \textsl{Gaia} DR3 angular resolution to $1.31^{+0.24}_{-0.29}$ arcsec. We also provide the $\xi$ -parameter for our IMF to be $0.5075_{-0.0051}^{+0.0112}$ for the users to compare our result with other work.

</div>

<div id="div_fig1">

<img src="tmp_2506.12987/./Fig/GCNS_cmd.png" alt="Fig2" width="100%"/>

**Figure 2. -** colour-magnitude diagram for stars in the 100-pc solar neighbourhood. The scatter with gray colour map illustrates the density distribution for all GCNS sample, overlapped with the object density distribution resulting from our sample selection procedure in Sec.\ref{sec:MS sample}. Black dashed lines denote the selection standards for Main-sequence stars in \citet{penoyre2022}, while black dotted lines denote the iso-mass lines for $0.15 \mathrm{M_{\odot}}$ and unevolved $0.15 \mathrm{M_{\odot}}$ stars from PARSEC model. (*fig:GCNS_cmd*)

</div>
<div id="div_fig2">

<img src="tmp_2506.12987/./Fig/emcee_corner_lamost_mdf_nuting.png" alt="Fig11" width="100%"/>

**Figure 11. -** The posterior distribution of the model parameters with our main sample described in Fig.\ref{fig:GCNS_cmd}. (*fig:mcmc*)

</div>
<div id="div_fig3">

<img src="tmp_2506.12987/./Fig/Completeness_exp.png" alt="Fig1" width="100%"/>

**Figure 1. -** Completeness limit as a function of absolute magnitude in the G-band. (*fig:completeness*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.12987"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

528  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
